In [1]:
import pandas as pd
import re
import numpy as np
import dill

from tqdm import tqdm
from collections import defaultdict, Counter

import warnings
warnings.filterwarnings("ignore")

# MIMIC IV

In [11]:
PATIENTS = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/PATIENTS.csv.gz')
ADMISSIONS = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/ADMISSIONS.csv.gz')
ICUSTAYS = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/icu/ICUSTAYS.csv.gz')

ADMISSIONS.columns = ADMISSIONS.columns.str.upper()
ICUSTAYS.columns = ICUSTAYS.columns.str.upper()
PATIENTS.columns = PATIENTS.columns.str.upper()

In [12]:
def MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS):
    # Select relevant columns for ADMISSIONS, ICUSTAYS, and PATIENTS
    ADMISSIONS = ADMISSIONS[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE']]
    ICUSTAYS = ICUSTAYS[['HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']]
    PATIENTS['DOB'] = PATIENTS['ANCHOR_YEAR'] - PATIENTS['ANCHOR_AGE']
    PATIENTS = PATIENTS[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD','ANCHOR_YEAR_GROUP']]
    
    # Merge the datasets
    df = pd.merge(PATIENTS, ADMISSIONS, on='SUBJECT_ID', how='left')
    df = pd.merge(df, ICUSTAYS, on='HADM_ID', how='left')
    print(df.shape,len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Convert date columns to datetime
    date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOD']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    # Sort the dataframe by 'SUBJECT_ID' and 'INTIME'
    df = df.dropna(subset=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOB'])
    print('Drop time missing', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    df = df.sort_values(by=['SUBJECT_ID', 'INTIME'])

    # Create 'DOB' as a complete date by assuming January 1st for each year
    df['DOB'] = pd.to_datetime(df['DOB'].astype(str) + '-01-01')
    # Calculate 'AGE' based on 'ADMITTIME' and 'DOB', and adjust for ages above 89
    df['AGE'] = (df['ADMITTIME'] - df['DOB']).dt.days // 365.242
    df['AGE'] = df['AGE'].apply(lambda x: 91 if x > 89 else x)
    # Filter out patients under 18
    df = df[df['AGE'] >= 18]
    print('AGE >= 18',df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Select specific columns for the final dataframe
    df = df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DISCHTIME', 'OUTTIME', 
             'GENDER', 'DOB', 'DOD', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 
             'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'LOS', 'AGE']]
    # Create a flag for the first admission for each patient
    df['FIRST_HADM'] = df.groupby('SUBJECT_ID')['ADMITTIME'].transform(lambda x: x == x.min()).astype(int)
    # Create a flag for the first ICU stay for each patient-HADM combination
    df['FIRST_ICU'] = df.groupby(['SUBJECT_ID', 'HADM_ID'])['INTIME'].transform(lambda x: x == x.min()).astype(int)
    # keep first ICU
    df = df[df['FIRST_ICU'] == 1]
    print('FIRST_ICU', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Keep rows with LOS greater than 1; 
    df = df[df['LOS'] >= 1]
    print('LOS', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))

    df['DIEINHOSPITAL'] = ((df['ADMITTIME'] <= df['DOD']) & (df['DOD'] <= df['DISCHTIME'])).astype(int)
    df['DIEINICU'] = ((df['INTIME'] <= df['DOD']) & (df['DOD'] <= df['OUTTIME'])).astype(int)

    df['NEXT_ADMIT'] = df.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
    df['DAYS_TO_READMIT'] = (df['NEXT_ADMIT'] - df['ADMITTIME']).dt.days
    df['Readmission_30'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 30 else 0)
    df['Readmission_60'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 60 else 0)

    df['HOURS_FROM_ADMIT'] = (df['INTIME'] - df['ADMITTIME']).dt.total_seconds() / 3600
    df['ICU_within_12hr_of_admit'] = df['HOURS_FROM_ADMIT'].apply(lambda x: 1 if 0 <= x <= 12 else 0)

    ICUSTAYS = ICUSTAYS[ICUSTAYS.HADM_ID.isin(df.HADM_ID)]
    ICUSTAYS['Multiple_ICUs'] = (ICUSTAYS.groupby('HADM_ID')['ICUSTAY_ID']
                    .transform('count')
                    .gt(1)  
                    .astype(int)) 
    ICUSTAYS = ICUSTAYS[['HADM_ID','Multiple_ICUs']].drop_duplicates(keep='first')
    df = pd.merge(df,ICUSTAYS,on='HADM_ID',how='left')
    df.drop(columns=['NEXT_ADMIT', 'DAYS_TO_READMIT'], inplace=True)

    df['SUBJECT_ID'] = df['SUBJECT_ID'].astype(int)
    df['HADM_ID'] = df['HADM_ID'].astype(int)
    df['ICUSTAY_ID'] = df['ICUSTAY_ID'].astype(int)

    print('------------------------------------------------------------------------')
    print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
    print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
    print('DIEINICU:',Counter(df['DIEINICU']))
    print('Readmission_30:',Counter(df['Readmission_30']))
    print('Readmission_60:',Counter(df['Readmission_60']))
    print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
    print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

    return df

In [13]:
ADMISSIONS = ADMISSIONS.rename(columns={'ETHNICITY':'RACE'})
ICUSTAYS = ICUSTAYS.rename(columns={'STAY_ID':'ICUSTAY_ID'})

df = MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS)

(696419, 18) 364627 546029 94459
Drop time missing (94444, 18) 65355 85229 94444
AGE >= 18 (94444, 19) 65355 85229 94444
FIRST_ICU (85229, 20) 65355 85229 85229
LOS (67225, 20) 53715 67225 67225
------------------------------------------------------------------------
Final Data： (67225, 27) SUBJECT_ID: 53715 HADM_ID: 67225 ICUSTAY_ID: 67225
DIEINHOSPITAL: Counter({0: 59635, 1: 7590})
DIEINICU: Counter({0: 62220, 1: 5005})
Readmission_30: Counter({0: 64796, 1: 2429})
Readmission_60: Counter({0: 62993, 1: 4232})
ICU_within_12hr_of_admit: Counter({1: 47012, 0: 20213})
Multiple_ICUs: Counter({0: 61177, 1: 6048})


In [14]:
df.head(2)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ADMITTIME,INTIME,DISCHTIME,OUTTIME,GENDER,DOB,DOD,...,AGE,FIRST_HADM,FIRST_ICU,DIEINHOSPITAL,DIEINICU,Readmission_30,Readmission_60,HOURS_FROM_ADMIT,ICU_within_12hr_of_admit,Multiple_ICUs
0,10000690,25860671,37081114,2150-11-02 18:02:00,2150-11-02 19:37:00,2150-11-12 13:45:00,2150-11-06 17:03:17,F,2064-01-01,2152-01-30,...,86.0,1,1,0,0,0,0,1.583333,1,0
1,10001217,24597018,37067082,2157-11-18 22:56:00,2157-11-20 19:18:02,2157-11-25 18:00:00,2157-11-21 22:08:00,F,2102-01-01,NaT,...,55.0,1,1,0,0,0,0,44.367222,0,0


In [15]:
df.columns[df.isnull().any()]

Index(['DOD', 'DEATHTIME'], dtype='object')

In [16]:
df.GENDER.value_counts()

M    37990
F    29235
Name: GENDER, dtype: int64

In [17]:
df_female = df[df.GENDER == 'F']
print('---------------------------Female-------------------------------------')
print('Final  ———— ：',df_female.shape, 'SUBJECT_ID:',len(df_female.SUBJECT_ID.unique()),'HADM_ID:',len(df_female.HADM_ID.unique()), 'ICUSTAY_ID:',len(df_female.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df_female['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df_female['DIEINICU']))
print('Readmission_30:',Counter(df_female['Readmission_30']))
print('Readmission_60:',Counter(df_female['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df_female['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df_female['Multiple_ICUs']))

------------------------Female----------------------------
Final ———— ： (29235, 27) SUBJECT_ID: 23150 HADM_ID: 29235 ICUSTAY_ID: 29235
DIEINHOSPITAL: Counter({0: 25831, 1: 3404})
DIEINICU: Counter({0: 27023, 1: 2212})
Readmission_30: Counter({0: 28175, 1: 1060})
Readmission_60: Counter({0: 27401, 1: 1834})
ICU_within_12hr_of_admit: Counter({1: 20669, 0: 8566})
Multiple_ICUs: Counter({0: 26683, 1: 2552})


In [18]:
df.to_csv('D:/2025UTI/IV/MIMIC4_p.csv', index=False)

In [2]:
df = pd.read_csv('D:/2025UTI/IV/MIMIC4_p.csv')
print('------------------------------------------------------------------------')
print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df['DIEINICU']))
print('Readmission_30:',Counter(df['Readmission_30']))
print('Readmission_60:',Counter(df['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

------------------------------------------------------------------------
Final Data： (67225, 27) SUBJECT_ID: 53715 HADM_ID: 67225 ICUSTAY_ID: 67225
DIEINHOSPITAL: Counter({0: 59635, 1: 7590})
DIEINICU: Counter({0: 62220, 1: 5005})
Readmission_30: Counter({0: 64796, 1: 2429})
Readmission_60: Counter({0: 62993, 1: 4232})
ICU_within_12hr_of_admit: Counter({1: 47012, 0: 20213})
Multiple_ICUs: Counter({0: 61177, 1: 6048})


In [3]:
D_ICD_DIAGNOSES = pd.read_csv('D:/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')
D_ICD_DIAGNOSES.columns = D_ICD_DIAGNOSES.columns.str.upper()
D_ICD_DIAGNOSES['ICD_VERSION'] = 9
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES[['ICD9_CODE','ICD_VERSION','LONG_TITLE']]
D_ICD_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_ICD_DIAGNOSES.shape)

D_4_DIAGNOSES = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/D_ICD_DIAGNOSES.csv.gz')
D_4_DIAGNOSES.columns = D_4_DIAGNOSES.columns.str.upper()
D_4_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(D_4_DIAGNOSES.shape)

D_ICD_DIAGNOSES = pd.concat([D_ICD_DIAGNOSES,D_4_DIAGNOSES])
D_ICD_DIAGNOSES = D_ICD_DIAGNOSES.drop_duplicates(keep='first')
print(D_ICD_DIAGNOSES.shape)

diag_9_10 = pd.read_csv('/0_diag_9_10.csv')
diag_9_10.columns = ['ICD_CODE', 'ROOT']
diag_9_10 = diag_9_10[~diag_9_10.duplicated(subset=['ICD_CODE'], keep=False)]
diag_9_10 = diag_9_10.dropna(subset=['ROOT'])
print(diag_9_10.shape,len(diag_9_10.ICD_CODE.unique()))

(14567, 3)
(112107, 3)
(112114, 3)
(17208, 2) 17208


In [4]:
D_ICD_9 = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 9]
D_ICD_10 = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 10]
diag_9 = D_ICD_9[D_ICD_9.ICD_CODE.isin(diag_9_10.ICD_CODE)].ICD_CODE.values
diag_10 = D_ICD_10[D_ICD_10.ICD_CODE.isin(diag_9_10.ICD_CODE)].ICD_CODE.values
print(len(diag_9), len(diag_10))
intersection = set(diag_9) & set(diag_10)
print(len(intersection), intersection)

5342 11954
86 {'V2509', 'E8301', 'V549', 'V453', 'V133', 'V170', 'V580', 'V4960', 'V103', 'V902', 'V163', 'V489', 'V671', 'V161', 'V182', 'V667', 'E892', 'E8029', 'V183', 'V850', 'V061', 'V2501', 'V169', 'V091', 'V812', 'E8020', 'V153', 'V180', 'V452', 'E8352', 'V714', 'E8350', 'V9039', 'E8319', 'V451', 'V155', 'V560', 'V486', 'E8339', 'V851', 'E8389', 'V173', 'V252', 'E8341', 'V664', 'V672', 'V189', 'V454', 'E8310', 'V109', 'V610', 'E8351', 'E8330', 'V093', 'V011', 'E896', 'V694', 'V854', 'V2541', 'V692', 'E895', 'V9081', 'V789', 'V9089', 'V707', 'V653', 'E8840', 'V655', 'E8359', 'V160', 'V4981', 'V9010', 'V698', 'V090', 'V175', 'V222', 'V172', 'V171', 'V232', 'V230', 'V162', 'V151', 'V712', 'V652', 'V619', 'V235'}


In [5]:
diag_10 = list(set(diag_10)-set(intersection))
print(len(diag_10))

11868


In [6]:
only_diag_9_10 = diag_9_10[diag_9_10.ICD_CODE.isin(diag_9)]
only_diag_9_10.shape

(5340, 2)

In [7]:
del(diag_9_10)

In [8]:
PCOS_9 = ['2564']
PCOS_10 = ['E282']
PCOS_ids = list(PCOS_9+PCOS_10)
print('多囊性卵巢:', len(PCOS_ids), PCOS_ids)

Neoplasm_ovary_9 = ['1830', '1986', '220', '2362', 'V1043', 'V1641', 'V8402']
Neoplasm_ovary_10 = ['C56', 'C561', 'C562', 'C569', 'C796', 'C7960', 'C7961', 'C7962','D27', 'D270', 'D271', 'D279', 'D391', 'D3910', 'D3911', 'D3912','Z1273', 'Z1502', 'Z8041', 'Z8543']
Neoplasm_ovary_ids = list(Neoplasm_ovary_9 + Neoplasm_ovary_10)
print('卵巢肿瘤:', len(Neoplasm_ovary_9),len(Neoplasm_ovary_10),len(Neoplasm_ovary_ids),Neoplasm_ovary_ids)

Endometriosis_9 = ['6170', '6171', '6172', '6173', '6174', '6175', '6176', '6178','6179']
Endometriosis_10 = ['N80', 'N800', 'N8000', 'N8001', 'N8002', 'N801', 'N8010', 'N80101', 'N80102', 'N80103', 'N80109', 'N8011', 'N80111', 'N80112', 'N80113', 'N80119', 'N8012', 'N80121', 'N80122', 'N80123', 'N80129', 'N802', 'N8020', 'N80201', 'N80202', 'N80203', 'N80209', 'N8021', 'N80211', 'N80212', 'N80213', 'N80219', 'N8022', 'N80221', 'N80222', 'N80223', 'N80229', 'N803', 'N8030', 'N8031', 'N80311', 'N80312', 'N80319', 'N8032', 'N80321', 'N80322', 'N80329', 'N8033', 'N80331', 'N80332', 'N80333', 'N80339', 'N8034', 'N80341', 'N80342', 'N80343', 'N80349', 'N8035', 'N80351', 'N80352', 'N80353', 'N80359', 'N8036', 'N80361', 'N80362', 'N80363', 'N80369', 'N8037', 'N80371', 'N80372', 'N80373', 'N80379', 'N8038', 'N80381', 'N80382', 'N80383', 'N80389', 'N8039', 'N80391', 'N80392', 'N80399', 'N803A', 'N803A1', 'N803A2', 'N803A3', 'N803A9', 'N803B', 'N803B1', 'N803B2', 'N803B3', 'N803B9', 'N803C', 'N803C1', 'N803C2', 'N803C3', 'N803C9', 'N804', 'N8040', 'N8041', 'N8042', 'N805', 'N8050', 'N8051', 'N80511', 'N80512', 'N80519', 'N8052', 'N80521', 'N80522', 'N80529', 'N8053', 'N80531', 'N80532', 'N80539', 'N8054', 'N80541', 'N80542', 'N80549', 'N8055', 'N80551', 'N80552', 'N80559', 'N8056', 'N80561', 'N80562', 'N80569', 'N806', 'N808', 'N809', 'N80A', 'N80A0', 'N80A1', 'N80A2', 'N80A4', 'N80A41', 'N80A42', 'N80A43', 'N80A49', 'N80A5', 'N80A51', 'N80A52', 'N80A53', 'N80A59', 'N80A6', 'N80A61', 'N80A62', 'N80A63', 'N80A69', 'N80B', 'N80B1', 'N80B2', 'N80B3', 'N80B31', 'N80B32', 'N80B39', 'N80B4', 'N80B5', 'N80B6', 'N80C', 'N80C0', 'N80C1', 'N80C10', 'N80C11', 'N80C19', 'N80C2', 'N80C3', 'N80C4', 'N80C9', 'N80D', 'N80D0', 'N80D1', 'N80D2', 'N80D3', 'N80D4', 'N80D5', 'N80D6', 'N80D9']
Endometriosis_ids = list(Endometriosis_9 + Endometriosis_10)
print('子宫内膜异位:', len(Endometriosis_9), len(Endometriosis_10), len(Endometriosis_ids), Endometriosis_ids)

Leiomyoma_9 = ['2180', '2181', '2182', '2189']
Leiomyoma_10 = ['D25', 'D250', 'D251', 'D252', 'D259']
Leiomyoma_ids = list(Leiomyoma_9 + Leiomyoma_10)
print('平滑肌瘤:', len(Leiomyoma_9), len(Leiomyoma_10), len(Leiomyoma_ids), Leiomyoma_ids)

Va_uti_9 = ['5990', '61610', '61611', '6174', '6273', '77182', '99664', 'E8796', 'V1302']
Va_uti_10 = ['N390', 'N760', 'N761', 'N771', 'N804', 'N8040', 'N8041', 'N8042','N952', 'N99521', 'N99531', 'O0338', 'O0388', 'O0488', 'O0738',
 'O0883', 'O233', 'O2330', 'O2331', 'O2332', 'O2333', 'O234','O2340', 'O2341', 'O2342', 'O2343', 'O8613', 'O862', 'O8620',
 'O8629', 'P393', 'T835', 'T8351', 'T83518', 'T83518A', 'T83518D', 'T83518S', 'T8351XA', 'T8351XD', 'T8351XS', 'T8359', 'T83590',
 'T83590A', 'T83590D', 'T83590S', 'T83591', 'T83591A', 'T83591D','T83591S', 'T83593', 'T83593A', 'T83593D', 'T83593S', 'T83598',
 'T83598A', 'T83598D', 'T83598S', 'T8359XA', 'T8359XD', 'T8359XS','Y846', 'Z87440']
Va_uti_ids = list(Va_uti_9 + Va_uti_10)
print('尿道感染|阴道炎:', len(Va_uti_9), len(Va_uti_10), len(Va_uti_ids), Va_uti_ids)

check_AKI_9 = ['5845', '5846', '5847', '5848','5849','66930','66932','66934']
check_AKI_10 = ['N17','N170','N171','N172','N179']
AKI_ids = list(check_AKI_9 + check_AKI_10)
print('AKI:', len(check_AKI_9), len(check_AKI_10), len(AKI_ids), AKI_ids)

check_CKD_9 = ['5851', '5852', '5853', '5854', '5855','5859'] 
check_CKD_10 = ['N18','N181','N182', 'N183', 'N1830', 'N1831', 'N1832', 'N184', 'N185', 'N189']
CKD_ids = list(check_CKD_9 + check_CKD_10)
print('CKD:', len(check_CKD_9), len(check_CKD_10), len(CKD_ids), CKD_ids)

多囊性卵巢: 2 ['2564', 'E282']
卵巢肿瘤: 7 20 27 ['1830', '1986', '220', '2362', 'V1043', 'V1641', 'V8402', 'C56', 'C561', 'C562', 'C569', 'C796', 'C7960', 'C7961', 'C7962', 'D27', 'D270', 'D271', 'D279', 'D391', 'D3910', 'D3911', 'D3912', 'Z1273', 'Z1502', 'Z8041', 'Z8543']
子宫内膜异位: 9 177 186 ['6170', '6171', '6172', '6173', '6174', '6175', '6176', '6178', '6179', 'N80', 'N800', 'N8000', 'N8001', 'N8002', 'N801', 'N8010', 'N80101', 'N80102', 'N80103', 'N80109', 'N8011', 'N80111', 'N80112', 'N80113', 'N80119', 'N8012', 'N80121', 'N80122', 'N80123', 'N80129', 'N802', 'N8020', 'N80201', 'N80202', 'N80203', 'N80209', 'N8021', 'N80211', 'N80212', 'N80213', 'N80219', 'N8022', 'N80221', 'N80222', 'N80223', 'N80229', 'N803', 'N8030', 'N8031', 'N80311', 'N80312', 'N80319', 'N8032', 'N80321', 'N80322', 'N80329', 'N8033', 'N80331', 'N80332', 'N80333', 'N80339', 'N8034', 'N80341', 'N80342', 'N80343', 'N80349', 'N8035', 'N80351', 'N80352', 'N80353', 'N80359', 'N8036', 'N80361', 'N80362', 'N80363', 'N80369',

In [9]:
specific_need = list(pd.unique(AKI_ids + CKD_ids + PCOS_ids + Neoplasm_ovary_ids + Endometriosis_ids + Leiomyoma_ids + Va_uti_ids))
print(len(specific_need),len(D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(specific_need)].ICD_CODE.unique()))

319 319


In [10]:
specific_need = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(specific_need)]
specific_need.head()

,ICD_CODE,ICD_VERSION,ICD_TEXT
1608,2564,9,Polycystic ovaries
1721,2362,9,Neoplasm of uncertain behavior of ovary
2043,1830,9,Malignant neoplasm of ovary
2151,1986,9,Secondary malignant neoplasm of ovary
2739,2180,9,Submucous leiomyoma of uterus


In [11]:
matched_9 = list(pd.unique(list(specific_need[specific_need.ICD_VERSION == 9].ICD_CODE.values)))
matched_10 = list(pd.unique(list(specific_need[specific_need.ICD_VERSION == 10].ICD_CODE.values)))
print(len(matched_9),len(matched_10))

43 276


In [12]:
matched_9_isin = only_diag_9_10[only_diag_9_10.ICD_CODE.isin(matched_9)].ICD_CODE.values #需要的label9，有对应的ICD10
matched_9_notin = list(set(matched_9)-set(matched_9_isin))  #需要的label9，没有对应的ICD10
print(len(matched_9_isin),len(matched_9_notin))

matched_10_add = list(pd.unique(list(matched_10) + list(pd.unique(only_diag_9_10[only_diag_9_10.ICD_CODE.isin(matched_9)].ROOT.values))))
print(len(matched_10_add))

print('需要单独纳入的ICD9：',len(matched_9_notin))
print('需要纳入的ICD10：',len(matched_10_add))

33 10
286
需要单独纳入的ICD9： 10
需要纳入的ICD10： 286


In [13]:
# 检查高频 diagnoses
def diag_9_10_check(df,diag_9_10,icds,dataset,n):
    if dataset == 'MIMICIV':
        print(dataset, 'ICD_VERSION:',Counter(df.ICD_VERSION))
    else:
        df['ICD_VERSION'] = 9
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))

    d_9 = df[df.ICD_VERSION == 9]
    d_10 = df[df.ICD_VERSION == 10]
    print(d_9.shape,d_10.shape)
    
    po_9 = d_9[d_9.ICD_CODE.isin(icds[icds.ICD_VERSION ==9].ICD_CODE)]
    po_10 = d_10[d_10.ICD_CODE.isin(icds[icds.ICD_VERSION ==10].ICD_CODE)]
    print('包含相应ICD9：',po_9.shape,'包含相应ICD10：',po_10.shape)
    print('存在的ICD9个数：',len(po_9.ICD_CODE.unique()),'存在的ICD10个数：',len(po_10.ICD_CODE.unique()))

    # 全部的CODE
    all_9code = po_9.ICD_CODE.value_counts()
    all_9code = all_9code[all_9code>n].index
    print('全部出现频率高的9code:', len(all_9code), all_9code)
    all_10code = po_10.ICD_CODE.value_counts()
    all_10code = all_10code[all_10code>n].index
    print('全部出现频率高的10code:', len(all_10code), all_10code)
    
    # 存在9-10对应的CODE，看看个数
    cc_9code = po_9[po_9.ICD_CODE.isin(diag_9_10[diag_9_10.ICD_CODE.isin(po_9.ICD_CODE.unique())].ICD_CODE)].ICD_CODE.value_counts()
    cc_9code = cc_9code[cc_9code>n].index
    print('在diag_9_10出现频率高的9code:', len(cc_9code), cc_9code)
    print('没有在diag_9_10的9code:', len(list(set(all_9code) - set(cc_9code))), list(set(all_9code) - set(cc_9code)))
    cc_9code_10 = list(diag_9_10[diag_9_10.ICD_CODE.isin(cc_9code)].ROOT.unique())
    print('对应的10code：',len(cc_9code_10),cc_9code_10)

    cc_10code = po_10[po_10.ICD_CODE.isin(diag_9_10[diag_9_10.ROOT.isin(po_10.ICD_CODE.unique())].ICD_CODE)].ICD_CODE.value_counts()
    cc_10code = cc_10code[cc_10code>n]
    print('在diag_9_10出现频率高的10code:', len(cc_10code.index), cc_10code.index)

    print('-----------------------------Final Screen-----------------------------')
    print('全部的ICD_10：',len(pd.unique(list(all_10code)+list(cc_9code_10))),list(pd.unique(list(all_10code)+list(cc_9code_10))))
    print('没有对应的9_code:：',len(list(set(all_9code) - set(cc_9code))), list(set(all_9code) - set(cc_9code)))

    return list(pd.unique(list(all_10code)+list(cc_9code_10))), list(set(all_9code) - set(cc_9code))

In [14]:
IV_diagnoses_raw = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/diagnoses_icd.csv.gz')
IV_diagnoses_raw.columns = IV_diagnoses_raw.columns.str.upper()
IV_diagnoses_raw = IV_diagnoses_raw[['SUBJECT_ID', 'HADM_ID', 'ICD_CODE','ICD_VERSION']]
print(IV_diagnoses_raw.shape, len(IV_diagnoses_raw.HADM_ID.unique()),len(IV_diagnoses_raw.ICD_CODE.unique()))

(6364488, 4) 545497 28562


In [15]:
#全部的code
D_10code, D_9code = diag_9_10_check(IV_diagnoses_raw,only_diag_9_10,D_ICD_DIAGNOSES,'MIMICIV',3000)

MIMICIV ICD_VERSION: Counter({10: 3455747, 9: 2908741})
(2908741, 4) (3455747, 4)
包含相应ICD9： (2908741, 4) 包含相应ICD10： (3455747, 4)
存在的ICD9个数： 9143 存在的ICD10个数： 19440
全部出现频率高的9code: 186 Index(['4019', '2724', '53081', '25000', '42731', '4280', '311', '41401',
       'V1582', '5849',
       ...
       'V667', 'V4502', '78321', 'V462', '5363', '29410', '70703', '66411',
       'V433', 'V5811'],
      dtype='object', length=186)
全部出现频率高的10code: 209 Index(['E785', 'I10', 'Z87891', 'K219', 'F329', 'I2510', 'F419', 'N179',
       'Z20822', 'Z7901',
       ...
       'I132', 'G936', 'J441', 'K449', 'E11649', 'K921', 'X58XXXA', 'K3184',
       'K7030', 'Z8619'],
      dtype='object', length=209)
在diag_9_10出现频率高的9code: 163 Index(['4019', '2724', '53081', '25000', '42731', '4280', '311', '41401',
       'V1582', '5849',
       ...
       'V08', '2639', '58381', 'V667', 'V4502', '78321', '29410', '5363',
       '70703', 'V5811'],
      dtype='object', length=163)
没有在diag_9_10的9code: 23 ['66411', 'E93

In [16]:
print(len(D_10code),len(D_9code))

296 23


In [18]:
d_10_iii = ['I95', 'E87', 'R10', 'F10', 'Z21', 'R68', 'J44', 'F30', 'K70', 'J69', 'I50', 'J98', 'N17', 'K92', 'R09', 'I47', 'D64', 'I25', 'M15', 'M81', 'R58', 'F41', 'I10', 'I34', 'I21', 'N18', 'I12', 'E11', 'Z51', 'K22', 'G93', 'G35', 'J45', 'F32', 'M79', 'Z91', 'K59', 'R45', 'I20', 'G43', 'G61', 'E78', 'E66', 'Z95', 'C33', 'J18', 'J41', 'R69', 'R00', 'T82', 'Q20', 'R19', 'K80', 'D50', 'A04', 'F19', 'B37', 'E89', 'N39', 'T78', 'Z00', 'K56', 'K57', 'K29', 'R78', 'D69', 'I42', 'I27', 'G89', 'B15', 'A40', 'T81', 'E83', 'G47', 'D66', 'E44', 'F05', 'K66', 'B95', 'K76', 'K91', 'N28', 'M10', 'L89', 'I44', 'I69', 'J15', 'Z85', 'I24', 'I66', 'G81', 'H40', 'Z81', 'G44', 'K75', 'I85', 'I61', 'K85', 'N23', 'K41', 'D70', 'K31', 'J93', 'N40', 'H35', 'C79', 'I31', 'I33', 'G40', 'M48', 'I73', 'M60', 'F04', 'L03', 'G60', 'S06', 'G30', 'I82', 'S12', 'I26', 'M06', 'E22', 'S27', 'C78', 'G97', 'G20', 'J84', 'I67', 'I60', 'D61', 'I71', 'B20', 'J43', 'Z16', 'I70', 'C22', 'I07', 'R06', 'C61', 'I62', 'N05', 'I08', 'C82', 'N13', 'J95', 'S01', 'M32', 'Z23']
len(d_10_iii)

148

In [19]:
d_9_iii = ['E9342', 'V1254', 'V433', 'E8790', 'V4364', 'E8859', 'E9320', '76519', 'V502', '7470', 'V290', '99731', 'V1259', '76527', '7742', '6930', 'E9478', '7756', '76517', 'E8889', 'V3000', '76516', '77081', 'E8490', 'V293', 'V422', '9974', '86121', '7464', '9973', '43310', '7793', 'E8786', 'E8781', 'V3101', 'E8788', '39891', 'E8498', 'E8780', 'V4365', '7746', 'V420', '769', '9971', '76518', 'E8809', '7661', '2113', 'V441', 'V1251', 'V8741', '77089', '5570', 'E8798', 'E9331', '9975', 'E8497', '7706', '99702', '76528', 'V3001', 'V440', 'V462', 'E8782', '76526', '77981', '7766', '77989']
len(d_9_iii)

68

In [20]:
d_10_iii_iv = list(pd.unique(d_10_iii + D_10code))
d_9_iii_iv = list(pd.unique(d_9_iii + D_9code))
print(len(d_10_iii_iv),len(d_9_iii_iv))

356 75


In [21]:
del(D_10code, D_9code,d_10_iii,d_9_iii)

In [24]:
label_9 = PCOS_9 + Neoplasm_ovary_9 + Endometriosis_9 + Leiomyoma_9 + Va_uti_9 + check_AKI_9 + check_CKD_9
label_10 = PCOS_10 + Neoplasm_ovary_10 + Endometriosis_10 + Leiomyoma_10 + Va_uti_10 + check_AKI_10 + check_CKD_10
print(len(label_9), len(label_10))

44 280


In [25]:
print('抛开用于分组的标签，最终纳入的初始 Diag：')
Final_9 = list(pd.unique(d_9_iii_iv + matched_9_notin))
Final_9 = list(set(Final_9)-set(['E8809','E8889', 'V270', 'V4364', 'V446', 'V600','E894']))
Final_10 = list(pd.unique(d_10_iii_iv + matched_10_add))
Final_10 = list(set(Final_10)-set(['E8339', 'E8342','N40', 'N400', 'N401', 'Z8546','V433','V462','V474','V536','V556','V290', 'V293', 'V420', 'V422', 'V433','V440', 'V441', 'V462', 'V502']))

print('Final_9：',len(Final_9))
print('Final_10：',len(Final_10))

抛开用于分组的标签，最终纳入的初始 Diag：
Final_9： 80
Final_10： 623


In [26]:
len(pd.unique(Final_9+Final_10))

703

In [29]:
# tt = D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(pd.unique(Final_9+Final_10))]
# print(tt.shape)
# tt[tt.duplicated(subset=['ICD_CODE'],keep=False)]

In [27]:
len(pd.unique(Final_9+Final_10+label_9+label_10))

736

In [30]:
# 定点取 ICD9 和 ICD10 对应的
def get_diag(df,Final_9,Final_10,label_9,label_10,dataset,patients,only_diag_9_10):
    if dataset == 'MIMICIV':
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))
    else:
        df['ICD_VERSION'] = 9
        print(dataset,'ICD_VERSION:',Counter(df.ICD_VERSION))
        
    df_diag_9 = df[df.ICD_VERSION == 9]
    df_diag_10 = df[df.ICD_VERSION == 10]
    print('ICD 9:',df_diag_9.shape, 'ICD 10:',df_diag_10.shape)

    # ----------------------------------对所有包括label列进行处理
    # 对没有mapped的 ICD_9进行处理
    df_diag_9_add = df_diag_9[df_diag_9.ICD_CODE.isin(list(pd.unique(Final_9+label_9)))] #一定要单独加进去的 icd9
    df_diag_9_mapped = df_diag_9[~(df_diag_9.ICD_CODE.isin(Final_9))] #找有映射的
    df_diag_9_mapped = pd.merge(df_diag_9_mapped,only_diag_9_10,on='ICD_CODE',how='left')
    df_diag_9_mapped = df_diag_9_mapped[df_diag_9_mapped.ROOT.isin(list(pd.unique(Final_10+label_10)))]
    print('补充的ICD9：', len(df_diag_9_add.ICD_CODE.unique()))
    print('9_mapped：', len(df_diag_9_mapped.ICD_CODE.unique()), len(df_diag_9_mapped.ROOT.unique()))

    # 对 ICD_10 进行处理
    df_diag_10 = df_diag_10[df_diag_10.ICD_CODE.isin(list(pd.unique(Final_10+label_10)))]
    print('ICD_10:', len(df_diag_10.ICD_CODE.unique()))

    all_df_diag = pd.concat([df_diag_9_mapped,df_diag_9_add])
    all_df_diag = pd.concat([all_df_diag,df_diag_10])
    all_df_diag['ROOT'] = all_df_diag['ROOT'].fillna(all_df_diag['ICD_CODE'])
    all_df_diag = all_df_diag.drop_duplicates(subset=['HADM_ID', 'ICD_CODE', 'ICD_VERSION', 'ROOT'], keep='first')
    print(all_df_diag.shape, len(all_df_diag.HADM_ID.unique()), len(all_df_diag.ICD_CODE.unique()), len(all_df_diag.ROOT.unique()))
    print(len(list(all_df_diag.ROOT.unique())))

    all_df_diag = all_df_diag[all_df_diag.HADM_ID.isin(patients.HADM_ID)]
    print(all_df_diag.shape, len(all_df_diag.HADM_ID.unique()), len(all_df_diag.ICD_CODE.unique()), len(all_df_diag.ROOT.unique()))
    print(len(list(all_df_diag.ROOT.unique())))

    # Group by subject and admission ID to list ICD codes per ROOT category
    all_df_diag = all_df_diag.groupby(['SUBJECT_ID', 'HADM_ID', 'ROOT'])['ICD_CODE'].apply(list).reset_index()
    # Pivot the grouped data to create a matrix for HADM_ID and ROOT categories
    all_df_diag = all_df_diag.pivot(index='HADM_ID', columns='ROOT', values='ICD_CODE')
    all_df_diag = all_df_diag.reset_index()
    print(all_df_diag.shape,len(all_df_diag.HADM_ID.unique()))
    
    all_df_diag.loc[:, all_df_diag.columns != 'HADM_ID'] = all_df_diag.loc[:, all_df_diag.columns != 'HADM_ID'].notna().astype(int)
    zero_cols = all_df_diag.columns[all_df_diag.eq(0).all()].tolist()
    zero_cols = list(set(zero_cols)-set(label_9+label_10))
    all_df_diag = all_df_diag.drop(columns=zero_cols)
    print(all_df_diag.shape,len(all_df_diag.HADM_ID.unique()))

    all_df_diag.columns = [
    ('Diag_9_' + col if col in (Final_9 + label_9) 
     else 'Diag_10_' + col if col in (Final_10 + label_10) 
     else col)
    for col in all_df_diag.columns]

    return all_df_diag

In [31]:
all_df_diag = get_diag(IV_diagnoses_raw,Final_9,Final_10,label_9,label_10,'MIMICIV',df,only_diag_9_10)

MIMICIV ICD_VERSION: Counter({10: 3455747, 9: 2908741})
ICD 9: (2908741, 4) ICD 10: (3455747, 4)
补充的ICD9： 86
9_mapped： 2106 160
ICD_10: 281
(4472422, 5) 529642 2440 504
504
(892148, 5) 66981 2336 482
482
(66981, 483) 66981
(66981, 483) 66981


In [32]:
all_df_diag.head()

,HADM_ID,Diag_9_1830,Diag_9_1986,Diag_9_2113,Diag_9_2180,Diag_9_2181,Diag_9_2182,Diag_9_2189,Diag_9_220,Diag_9_2362,...,Diag_10_Z9221,Diag_10_Z923,Diag_10_Z95,Diag_10_Z950,Diag_10_Z951,Diag_10_Z952,Diag_10_Z955,Diag_10_Z9884,Diag_10_Z992,Diag_10_Z9981
0,20000147,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20001305,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,20001361,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20001494,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,20001770,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
print(len(label_9),len(label_10))
print(len(pd.unique(label_9)),len(pd.unique(label_10)))
now_l_9 = list(pd.unique(label_9))
now_l_9 = ['Diag_9_'+i for i in now_l_9]
now_l_10 = list(pd.unique(label_10))
now_l_10 = ['Diag_10_'+i for i in now_l_10]
print(len(now_l_9),len(now_l_10))

# 有些feature压根没有sample
existing_l_9 = list(set(now_l_9) & set(all_df_diag.columns))
existing_l_10 = list(set(now_l_10) & set(all_df_diag.columns))
print(len(existing_l_9),len(existing_l_10))

# 具体到什么标签
# a_d = [i.split('_')[-1] for i in all_df_diag.columns if 'Diag_' in i]
existing_AKI = list(set(['Diag_9_'+i for i in check_AKI_9]+['Diag_10_'+i for i in check_AKI_10]) & set(all_df_diag.columns))
existing_CKD = list(set(['Diag_9_'+i for i in check_CKD_9]+['Diag_10_'+i for i in check_CKD_10]) & set(all_df_diag.columns))
existing_PCOS = list(set(['Diag_9_'+i for i in PCOS_9]+['Diag_10_'+i for i in PCOS_10]) & set(all_df_diag.columns))
existing_Neoplasm_ovary = list(set(['Diag_9_'+i for i in Neoplasm_ovary_9]+['Diag_10_'+i for i in Neoplasm_ovary_10]) & set(all_df_diag.columns))
existing_Endometriosis = list(set(['Diag_9_'+i for i in Endometriosis_9]+['Diag_10_'+i for i in Endometriosis_10]) & set(all_df_diag.columns))
existing_Leiomyoma = list(set(['Diag_9_'+i for i in Leiomyoma_9]+['Diag_10_'+i for i in Leiomyoma_10]) & set(all_df_diag.columns))
existing_Va_uti = list(set(['Diag_9_'+i for i in Va_uti_9]+['Diag_10_'+i for i in Va_uti_10]) & set(all_df_diag.columns))
print(len(existing_AKI),len(existing_CKD),len(existing_PCOS),len(existing_Neoplasm_ovary),len(existing_Endometriosis),len(existing_Leiomyoma),len(existing_Va_uti))

44 280
43 276
43 276
35 55
10 16 2 19 14 9 20


In [34]:
all_df_diag['Group_AKI'] = (all_df_diag[existing_AKI] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_AKI']), Counter(all_df_diag['Group_AKI']))
print('\n')

all_df_diag['Group_CKD'] = (all_df_diag[existing_CKD] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_CKD']), Counter(all_df_diag['Group_CKD']))
print('\n')

all_df_diag['Group_PCOS'] = (all_df_diag[existing_PCOS] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_PCOS']))
print(len(existing_PCOS), existing_PCOS)
for c in existing_PCOS:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Neoplasm_ovary'] = (all_df_diag[existing_Neoplasm_ovary] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Neoplasm_ovary']))
print(len(existing_Neoplasm_ovary), existing_Neoplasm_ovary)
for c in existing_Neoplasm_ovary:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Endometriosis'] = (all_df_diag[existing_Endometriosis] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Endometriosis']))
print(len(existing_Endometriosis), existing_Endometriosis)
for c in existing_Endometriosis:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Leiomyoma'] = (all_df_diag[existing_Leiomyoma] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Leiomyoma']))
print(len(existing_Leiomyoma), existing_Leiomyoma)
for c in existing_Leiomyoma:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

all_df_diag['Group_Va_uti'] = (all_df_diag[existing_Va_uti] == 1).any(axis=1).astype(int)
print(Counter(all_df_diag['Group_Va_uti']))
print(len(existing_Va_uti), existing_Va_uti)
for c in existing_Va_uti:
    print(c,D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE==c.split('_')[-1]].ICD_TEXT.values[0],Counter(all_df_diag[c]))
print('\n')

Counter({0: 45562, 1: 21419}) Counter({0: 45562, 1: 21419})


Counter({0: 54014, 1: 12967}) Counter({0: 54014, 1: 12967})


Counter({0: 66910, 1: 71})
2 ['Diag_9_2564', 'Diag_10_E282']
Diag_9_2564 Polycystic ovaries Counter({0: 66956, 1: 25})
Diag_10_E282 Polycystic ovarian syndrome Counter({0: 66935, 1: 46})


Counter({0: 66635, 1: 346})
19 ['Diag_10_C7960', 'Diag_10_Z8041', 'Diag_10_D27', 'Diag_10_C562', 'Diag_10_C569', 'Diag_9_1986', 'Diag_10_C7962', 'Diag_10_Z8543', 'Diag_9_1830', 'Diag_9_V1641', 'Diag_10_D270', 'Diag_9_2362', 'Diag_9_V1043', 'Diag_10_D3912', 'Diag_10_C56', 'Diag_10_C7961', 'Diag_10_D271', 'Diag_9_220', 'Diag_10_C561']
Diag_10_C7960 Secondary malignant neoplasm of unspecified ovary Counter({0: 66972, 1: 9})
Diag_10_Z8041 Family history of malignant neoplasm of ovary Counter({0: 66961, 1: 20})
Diag_10_D27 Benign neoplasm of ovary Counter({0: 66965, 1: 16})
Diag_10_C562 Malignant neoplasm of left ovary Counter({0: 66963, 1: 18})
Diag_10_C569 Malignant neoplasm of uns

In [35]:
all_df_diag.head()

,HADM_ID,Diag_9_1830,Diag_9_1986,Diag_9_2113,Diag_9_2180,Diag_9_2181,Diag_9_2182,Diag_9_2189,Diag_9_220,Diag_9_2362,...,Diag_10_Z9884,Diag_10_Z992,Diag_10_Z9981,Group_AKI,Group_CKD,Group_PCOS,Group_Neoplasm_ovary,Group_Endometriosis,Group_Leiomyoma,Group_Va_uti
0,20000147,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20001305,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,20001361,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,20001494,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20001770,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [36]:
all_df_diag.shape

(66981, 490)

In [37]:
all_df_diag.to_csv('D:/2025UTI/IV/IV_all_diag.csv',index=False)

### 检查III和IV的诊断信息，并把一样意思的icd合并

In [38]:
all_df_diag = pd.read_csv('D:/2025UTI/IV/IV_all_diag.csv')

In [45]:
existing_labels = existing_AKI + existing_CKD + existing_PCOS + existing_Neoplasm_ovary + existing_Endometriosis + existing_Leiomyoma + existing_Va_uti
len(existing_labels)

90

In [46]:
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=existing_labels, errors='ignore')
all_df_diag.shape

(66981, 490)


(66981, 400)

In [47]:
iv_ids = [i for i in all_df_diag.columns if 'Diag_' in i]
len(iv_ids)

392

In [48]:
iii_ids = ['Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Diag_10_F10', 'Diag_10_F19', 'Diag_10_F30', 'Diag_10_F32', 'Diag_10_F41', 'Diag_10_F43', 'Diag_10_G20', 'Diag_10_G30', 'Diag_10_G35', 'Diag_10_G40', 'Diag_10_G43', 'Diag_10_G44', 'Diag_10_G47', 'Diag_10_G60', 'Diag_10_G61', 'Diag_10_G81', 'Diag_10_G89', 'Diag_10_G93', 'Diag_10_G97', 'Diag_10_H35', 'Diag_10_H40', 'Diag_10_I07', 'Diag_10_I08', 'Diag_10_I10', 'Diag_10_I12', 'Diag_10_I20', 'Diag_10_I21', 'Diag_10_I24', 'Diag_10_I25', 'Diag_10_I26', 'Diag_10_I27', 'Diag_10_I31', 'Diag_10_I33', 'Diag_10_I34', 'Diag_10_I42', 'Diag_10_I44', 'Diag_10_I47', 'Diag_10_I50', 'Diag_10_I60', 'Diag_10_I61', 'Diag_10_I62', 'Diag_10_I66', 'Diag_10_I67', 'Diag_10_I69', 'Diag_10_I70', 'Diag_10_I71', 'Diag_10_I73', 'Diag_10_I82', 'Diag_10_I85', 'Diag_10_I95', 'Diag_10_J15', 'Diag_10_J18', 'Diag_10_J41', 'Diag_10_J43', 'Diag_10_J44', 'Diag_10_J45', 'Diag_10_J69', 'Diag_10_J84', 'Diag_10_J93', 'Diag_10_J95', 'Diag_10_J98', 'Diag_10_K22', 'Diag_10_K29', 'Diag_10_K31', 'Diag_10_K41', 'Diag_10_K56', 'Diag_10_K57', 'Diag_10_K59', 'Diag_10_K66', 'Diag_10_K70', 'Diag_10_K75', 'Diag_10_K76', 'Diag_10_K80', 'Diag_10_K85', 'Diag_10_K91', 'Diag_10_K92', 'Diag_10_L03', 'Diag_10_L89', 'Diag_10_M06', 'Diag_10_M10', 'Diag_10_M15', 'Diag_10_M32', 'Diag_10_M48', 'Diag_10_M60', 'Diag_10_M79', 'Diag_10_M81', 'Diag_10_N05', 'Diag_10_N13', 'Diag_10_N23', 'Diag_10_N28', 'Diag_10_N39', 'Diag_10_N72', 'Diag_10_N92', 'Diag_10_O75', 'Diag_10_Q20', 'Diag_10_R00', 'Diag_10_R06', 'Diag_10_R09', 'Diag_10_R10', 'Diag_10_R19', 'Diag_10_R45', 'Diag_10_R58', 'Diag_10_R63', 'Diag_10_R68', 'Diag_10_R69', 'Diag_10_R78', 'Diag_10_S01', 'Diag_10_S06', 'Diag_10_S12', 'Diag_10_S27', 'Diag_10_T78', 'Diag_10_T81', 'Diag_10_T82', 'Diag_9_V1251', 'Diag_9_V1254', 'Diag_9_V1255', 'Diag_9_V1259', 'Diag_9_V420', 'Diag_9_V422', 'Diag_9_V433', 'Diag_9_V4365', 'Diag_9_V440', 'Diag_9_V441', 'Diag_9_V462', 'Diag_9_V6284', 'Diag_9_V8741', 'Diag_10_Z00', 'Diag_10_Z16', 'Diag_10_Z21', 'Diag_10_Z23', 'Diag_10_Z51', 'Diag_10_Z80', 'Diag_10_Z81', 'Diag_10_Z85', 'Diag_10_Z87', 'Diag_10_Z91', 'Diag_10_Z95']
iv_ids = list(pd.unique(iv_ids+iii_ids))
len(iv_ids)

392

In [50]:
# print(iv_ids)

In [51]:
iv_ids_dict = pd.DataFrame()
for i in iv_ids:
    iv_ids_dict = pd.concat([iv_ids_dict, D_ICD_DIAGNOSES[(D_ICD_DIAGNOSES.ICD_CODE == i.split('_')[-1])&(D_ICD_DIAGNOSES.ICD_VERSION == int(i.split('_')[1]))]])
iv_ids_dict = iv_ids_dict.sort_values(by=['ICD_TEXT'])
duplicated = iv_ids_dict[iv_ids_dict.duplicated(subset=['ICD_TEXT'],keep=False)]

In [52]:
duplicated

,ICD_CODE,ICD_VERSION,ICD_TEXT
111932,Z9221,10,Personal history of antineoplastic chemotherapy
13394,V8741,9,Personal history of antineoplastic chemotherapy
111667,Z86711,10,Personal history of pulmonary embolism
13376,V1255,9,Personal history of pulmonary embolism
111670,Z8673,10,Personal history of transient ischemic attack ...
13375,V1254,9,Personal history of transient ischemic attack ...
26673,J18,10,"Pneumonia, unspecified organism"
26678,J189,10,"Pneumonia, unspecified organism"


In [53]:
dupi_ids = []
for i in pd.unique(duplicated['ICD_TEXT'].values):
    print('\n',i)
    
    # 找到当前 ICD_TEXT 的所有行
    this_rows = duplicated[duplicated['ICD_TEXT'] == i]
    
    # 构造列名列表：Diag_版本_编码
    this_cols = ('Diag_' + this_rows['ICD_VERSION'].astype(str) + '_' + this_rows['ICD_CODE'].astype(str)).tolist()
    dupi_ids.extend(this_cols)
    
    # 过滤出 all_df_diag 中确实存在的列
    this_cols_existing = [col for col in this_cols if col in all_df_diag.columns]
    
    # 打印计数信息
    for j in this_cols:
        if j in all_df_diag.columns:
            print(Counter(all_df_diag[j]))
        else:
            print(f"  Column {j} not found in all_df_diag for {i}")

    # 如果没有匹配的列，跳过这一轮
    if not this_cols_existing:
        print(f"  No matching columns in all_df_diag for {i}")
        continue

    # 聚合新的一列
    all_df_diag['Diag_' + i] = all_df_diag[this_cols_existing].eq(1).any(axis=1).astype(int)
    
    # 打印新列的计数
    print(Counter(all_df_diag['Diag_' + i]))



 Personal history of antineoplastic chemotherapy
Counter({0: 66455, 1: 526})
Counter({0: 66100, 1: 881})
Counter({0: 65574, 1: 1407})

 Personal history of pulmonary embolism
Counter({0: 65661, 1: 1320})
Counter({0: 66442, 1: 539})
Counter({0: 65122, 1: 1859})

 Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits
Counter({0: 64356, 1: 2625})
Counter({0: 65227, 1: 1754})
Counter({0: 62602, 1: 4379})

 Pneumonia, unspecified organism
Counter({0: 63506, 1: 3475})
Counter({0: 63672, 1: 3309})
Counter({0: 60197, 1: 6784})


In [54]:
## 删除重复列
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=dupi_ids, errors='ignore')
all_df_diag.shape

(66981, 404)


(66981, 396)

In [63]:
all_df_diag.shape

(66981, 396)

In [64]:
all_df_diag.to_csv('D:/2025UTI/IV/IV_all_diag.csv',index=False)

In [67]:
all_df_diag = pd.read_csv('D:/2025UTI/IV/IV_all_diag.csv')

In [68]:
f_iii = ['HADM_ID', 'Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Diag_10_F10', 'Diag_10_F19', 'Diag_10_F30', 'Diag_10_F32', 'Diag_10_F41', 'Diag_10_F43', 'Diag_10_G20', 'Diag_10_G30', 'Diag_10_G35', 'Diag_10_G40', 'Diag_10_G43', 'Diag_10_G44', 'Diag_10_G47', 'Diag_10_G60', 'Diag_10_G61', 'Diag_10_G81', 'Diag_10_G89', 'Diag_10_G93', 'Diag_10_G97', 'Diag_10_H35', 'Diag_10_H40', 'Diag_10_I07', 'Diag_10_I08', 'Diag_10_I10', 'Diag_10_I12', 'Diag_10_I20', 'Diag_10_I21', 'Diag_10_I24', 'Diag_10_I25', 'Diag_10_I26', 'Diag_10_I27', 'Diag_10_I31', 'Diag_10_I33', 'Diag_10_I34', 'Diag_10_I42', 'Diag_10_I44', 'Diag_10_I47', 'Diag_10_I50', 'Diag_10_I60', 'Diag_10_I61', 'Diag_10_I62', 'Diag_10_I66', 'Diag_10_I67', 'Diag_10_I69', 'Diag_10_I70', 'Diag_10_I71', 'Diag_10_I73', 'Diag_10_I82', 'Diag_10_I85', 'Diag_10_I95', 'Diag_10_J15', 'Diag_10_J41', 'Diag_10_J43', 'Diag_10_J44', 'Diag_10_J45', 'Diag_10_J69', 'Diag_10_J84', 'Diag_10_J93', 'Diag_10_J95', 'Diag_10_J98', 'Diag_10_K22', 'Diag_10_K29', 'Diag_10_K31', 'Diag_10_K41', 'Diag_10_K56', 'Diag_10_K57', 'Diag_10_K59', 'Diag_10_K66', 'Diag_10_K70', 'Diag_10_K75', 'Diag_10_K76', 'Diag_10_K80', 'Diag_10_K85', 'Diag_10_K91', 'Diag_10_K92', 'Diag_10_L03', 'Diag_10_L89', 'Diag_10_M06', 'Diag_10_M10', 'Diag_10_M15', 'Diag_10_M32', 'Diag_10_M48', 'Diag_10_M60', 'Diag_10_M79', 'Diag_10_M81', 'Diag_10_N05', 'Diag_10_N13', 'Diag_10_N23', 'Diag_10_N28', 'Diag_10_N39', 'Diag_10_N72', 'Diag_10_N92', 'Diag_10_O75', 'Diag_10_Q20', 'Diag_10_R00', 'Diag_10_R06', 'Diag_10_R09', 'Diag_10_R10', 'Diag_10_R19', 'Diag_10_R45', 'Diag_10_R58', 'Diag_10_R63', 'Diag_10_R68', 'Diag_10_R69', 'Diag_10_R78', 'Diag_10_S01', 'Diag_10_S06', 'Diag_10_S12', 'Diag_10_S27', 'Diag_10_T78', 'Diag_10_T81', 'Diag_10_T82', 'Diag_9_V1251', 'Diag_9_V1259', 'Diag_9_V420', 'Diag_9_V422', 'Diag_9_V433', 'Diag_9_V4365', 'Diag_9_V440', 'Diag_9_V441', 'Diag_9_V462', 'Diag_9_V6284', 'Diag_10_Z00', 'Diag_10_Z16', 'Diag_10_Z21', 'Diag_10_Z23', 'Diag_10_Z51', 'Diag_10_Z80', 'Diag_10_Z81', 'Diag_10_Z85', 'Diag_10_Z87', 'Diag_10_Z91', 'Diag_10_Z95', 'Group_AKI', 'Group_CKD', 'Group_PCOS', 'Group_Neoplasm_ovary', 'Group_Endometriosis', 'Group_Leiomyoma', 'Group_Va_uti', 'Diag_Personal history of antineoplastic chemotherapy', 'Diag_Personal history of pulmonary embolism', 'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits', 'Diag_Pneumonia, unspecified organism']
len(f_iii)

207

In [69]:
print(list(set(f_iii)-set(all_df_diag.columns)))

[]


In [72]:
oii = list(set(all_df_diag.columns)-set(f_iii+['Diag_10_A419','Diag_10_D62','Diag_10_R6521','Diag_10_Z794']))
oii_id = [i.split('_')[-1] for i in oii if 'Diag_' in i]
D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(oii_id)]#.ICD_TEXT.values

,ICD_CODE,ICD_VERSION,ICD_TEXT
13071,B1920,10,Unspecified viral hepatitis C without hepatic ...
13501,B9620,10,Unspecified Escherichia coli [E. coli] as the ...
13514,B9689,10,Other specified bacterial agents as the cause ...
14366,C787,10,Secondary malignant neoplasm of liver and intr...
14387,C7951,10,Secondary malignant neoplasm of bone
...,...,...,...
111985,Z952,10,Presence of prosthetic heart valve
111988,Z955,10,Presence of coronary angioplasty implant and g...
112088,Z9884,10,Bariatric surgery status
112102,Z992,10,Dependence on renal dialysis


In [73]:
print(oii)

['Diag_10_D61818', 'Diag_10_Z950', 'Diag_10_Z7901', 'Diag_10_G43909', 'Diag_10_G629', 'Diag_10_E039', 'Diag_10_I5032', 'Diag_10_I472', 'Diag_10_F32A', 'Diag_10_R627', 'Diag_10_Z87891', 'Diag_10_Z85828', 'Diag_10_K760', 'Diag_10_E860', 'Diag_10_I110', 'Diag_10_Z5111', 'Diag_10_Z86718', 'Diag_10_D696', 'Diag_10_E1151', 'Diag_10_I21A1', 'Diag_10_G8929', 'Diag_10_Z370', 'Diag_10_M069', 'Diag_10_J45909', 'Diag_10_F17200', 'Diag_10_I129', 'Diag_10_M1990', 'Diag_10_E861', 'Diag_10_I4891', 'Diag_10_R45851', 'Diag_10_Y832', 'Diag_10_I340', 'Diag_10_R1310', 'Diag_10_R338', 'Diag_10_R51', 'Diag_10_R079', 'Diag_10_Y848', 'Diag_10_Y929', 'Diag_10_I5023', 'Diag_10_M810', 'Diag_10_H409', 'Diag_10_R339', 'Diag_10_J441', 'Diag_10_Z951', 'Diag_10_G9341', 'Diag_10_R791', 'Diag_10_K7030', 'Diag_10_I509', 'Diag_10_D649', 'Diag_10_K3184', 'Diag_10_Z7902', 'Diag_10_I350', 'Diag_10_I8510', 'Diag_10_I951', 'Diag_10_M549', 'Diag_10_N186', 'Diag_10_E876', 'Diag_10_Z923', 'Diag_10_J690', 'Diag_10_I5022', 'Diag_10

In [74]:
## 对齐MIMIC III， 删除III没有的列
print(all_df_diag.shape)
all_df_diag = all_df_diag.drop(columns=oii, errors='ignore')
all_df_diag.shape

(66981, 396)


(66981, 211)

In [75]:
oii = list(set(all_df_diag.columns)-set(f_iii+['Diag_10_A419','Diag_10_D62','Diag_10_R6521','Diag_10_Z794']))
oii_id = [i.split('_')[-1] for i in oii if 'Diag_' in i]
D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_CODE.isin(oii_id)]#.ICD_TEXT.values

,ICD_CODE,ICD_VERSION,ICD_TEXT


In [76]:
all_df_diag.to_csv('D:/2025UTI/IV/IV_all_diag_matchIII.csv',index=False)

In [77]:
print(all_df_diag.columns.tolist())

['HADM_ID', 'Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_A419', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D62', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Dia

# Procedure

In [78]:
df = pd.read_csv('D:/2025UTI/IV/MIMIC4_p.csv')
print('------------------------------------------------------------------------')
print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df['DIEINICU']))
print('Readmission_30:',Counter(df['Readmission_30']))
print('Readmission_60:',Counter(df['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

------------------------------------------------------------------------
Final Data： (67225, 27) SUBJECT_ID: 53715 HADM_ID: 67225 ICUSTAY_ID: 67225
DIEINHOSPITAL: Counter({0: 59635, 1: 7590})
DIEINICU: Counter({0: 62220, 1: 5005})
Readmission_30: Counter({0: 64796, 1: 2429})
Readmission_60: Counter({0: 62993, 1: 4232})
ICU_within_12hr_of_admit: Counter({1: 47012, 0: 20213})
Multiple_ICUs: Counter({0: 61177, 1: 6048})


In [152]:
d_iii_icd_procedures = pd.read_csv('D:/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/d_icd_procedures.csv.gz')
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'ICD9_CODE':'ICD_CODE'})
d_iii_icd_procedures = d_iii_icd_procedures.rename(columns={'LONG_TITLE':'ICD_TEXT'})
d_iii_icd_procedures['ICD_VERSION'] = 9
d_iii_icd_procedures = d_iii_icd_procedures[['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']]
d_iii_icd_procedures['ICD_CODE'] = d_iii_icd_procedures['ICD_CODE'].astype(str)

d_iv_icd_procedures = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/d_icd_procedures.csv.gz')
d_iv_icd_procedures.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
d_iv_icd_procedures['ICD_CODE'] = d_iv_icd_procedures['ICD_CODE'].astype(str)

d_icd_procedures = pd.concat([d_iii_icd_procedures,d_iv_icd_procedures])
d_icd_procedures = d_icd_procedures.drop_duplicates(keep='first')

# 重复的text对应的ID
dd = d_icd_procedures[d_icd_procedures.duplicated(subset=['ICD_TEXT'],keep=False)].sort_values(by='ICD_TEXT')
dd = (
    dd.groupby('ICD_TEXT')
    .agg(
        final_ICD_CODE=('ICD_CODE', lambda x: max(x, key=len)),  # 取字符最长的ICD_CODE
        all_ICD_CODEs=('ICD_CODE', lambda x: list(x))            # 收集所有ICD_CODE
    )
    .reset_index()
)
#删除重复id，保留最长的id
d_icd_procedures = (
    d_icd_procedures.assign(code_len=d_icd_procedures['ICD_CODE'].str.len())
    .sort_values('code_len', ascending=False)
    .drop_duplicates(subset=['ICD_TEXT'], keep='first')
    .drop(columns='code_len')
)

p_map = {
    code: final 
    for final, codes in zip(dd['final_ICD_CODE'], dd['all_ICD_CODEs']) 
    for code in codes if code!=final
}

print(d_icd_procedures.ICD_VERSION.value_counts())
d_icd_procedures.head(2)

10    82534
9      3887
Name: ICD_VERSION, dtype: int64


,ICD_CODE,ICD_VERSION,ICD_TEXT
39513,0HPU8KZ,10,Removal of Nonautologous Tissue Substitute fro...
53718,0QW247Z,10,Revision of Autologous Tissue Substitute in Ri...


In [155]:
# pro的icd9和10也有重复的
print(d_icd_procedures.shape,len(d_icd_procedures.ICD_CODE.unique()))
d_icd_procedures[d_icd_procedures.duplicated('ICD_CODE',keep=False)]#.ICD_TEXT.values

(86421, 3) 86415


,ICD_CODE,ICD_VERSION,ICD_TEXT
25243,093,9,Other operations on lacrimal gland
25244,093,10,"Ear, Nose, Sinus, Control"
25152,090,9,Incision of lacrimal gland
25153,090,10,"Ear, Nose, Sinus, Alteration"
6422,031,9,Division of intraspinal nerve root
6423,031,10,"Upper Arteries, Bypass"
12755,045,10,"Lower Arteries, Destruction"
12754,045,9,Cranial or peripheral nerve graft
20192,067,10,"Lower Veins, Dilation"
20191,067,9,Excision of thyroglossal duct or tract


In [156]:
p_raw = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/procedures_icd.csv.gz')
p_raw.columns = p_raw.columns.str.upper()
p_raw = p_raw[['SUBJECT_ID', 'HADM_ID','ICD_CODE']]
p_raw['ICD_CODE'] = p_raw['ICD_CODE'].astype(str)
p_raw['ICD_CODE'] = p_raw['ICD_CODE'].replace(p_map)
p_raw = p_raw.drop_duplicates(keep='first')
print('IV_p_raw',p_raw.shape,len(p_raw.HADM_ID.unique()))

IV_p_raw (816489, 3) 287504


In [157]:
p_raw.head(2)

,SUBJECT_ID,HADM_ID,ICD_CODE
0,10000032,22595853,5491
1,10000032,22841357,5491


In [158]:
## 按照'ICD_VERSION', 'ICD_CODE'来统计
#tt = p_raw.value_counts(subset=['ICD_VERSION', 'ICD_CODE'])
# most_p = list(tt[tt>2000].index)
# most_p_ids = [item[1] for item in most_p]
# print(len(most_p),len(most_p_ids),len(pd.unique(most_p_ids)))

# 只按照ICD_CODE来统计
tt = p_raw.ICD_CODE.value_counts()
most_p_ids = list(tt[tt>2000].index)
print(len(most_p_ids))

71


In [159]:
iii_p = ['3893', '9604', '966', '9671', '9904', '3961', '9672', '9955', '8856', '3891', '3615', '9915', '8872', '3722', '3995', '9390', '3324', '3723', '4513', '9983', '640', '8853', '0331', '9907', '3612', '311', '9920', '3491', '3404', '0040', '4311', '3895', '3613', '3897', '5491', '3521', '8841', '0066', '9905', '3606', '3607', '8964', '3601', '3322', '3323', '3761', '3611', '0045', '0017', '8855', '9962', '4523', '387', '8842', '0014', '8659', '3950', '5459', '9910', '8847', '0159', '9960', '4443', '4516', '9605', '8848', '3721', '9607', '8622', '8852', '3845', '3522', '9925']
len(iii_p)

73

In [160]:
#included Procedures id
icds = [9604, 3722, 3723, 225966, 8856, 224270, 3995, 224385, 3615, 225402, 4513, 8872, 9390, 9904, 3891, 3893, 225401, 3897, 9915, 224264, 966, 9671, 9672, 221217, 3961, 221214, 225792, 225454, 225752, 227194, 3324]
icds = [str(i) for i in icds]
icds = list(pd.unique(icds + most_p_ids+iii_p))
len(icds)

124

In [161]:
icds = list(d_icd_procedures[d_icd_procedures.ICD_CODE.isin(icds)].ICD_CODE.values)
len(icds)

111

In [162]:
print(icds)

['0KQM0ZZ', 'B24BZZ4', 'B211YZZ', 'B2111ZZ', '5A1221Z', '5A1935Z', '5A1D70Z', '5A1955Z', '5A1945Z', '10E0XZZ', '3E0G76Z', '4A023N7', '3E04305', '3E0436Z', '10D00Z1', '10907ZC', '0W9G3ZX', '0W9G3ZZ', '0W9930Z', '02100Z9', '02H633Z', '009U3ZX', '03HY32Z', '02HV33Z', '0DJD8ZZ', '0DJ08ZZ', '0BH17EZ', '8162', '8952', '8964', '7359', '8872', '7309', '8842', '8847', '8848', '8852', '8853', '8855', '8856', '8841', '8938', '5491', '5459', '3324', '3323', '3322', '3491', '3404', '5185', '3522', '3521', '7569', '4513', '4516', '4443', '3995', '4523', '3761', '4311', '3891', '3893', '3895', '3897', '3722', '3723', '3845', '0159', '3961', '3899', '3950', '0040', '0017', '0045', '9960', '0014', '9962', '0066', '0741', '3611', '3607', '3612', '3613', '3615', '3606', '3721', '0331', '9983', '9390', '8622', '8744', '8659', '8051', '9920', '9925', '9904', '9905', '9915', '9907', '9910', '9955', '9607', '9671', '9672', '9605', '9604', '387', '311', '966', '734', '640']


In [163]:
p_raw = p_raw[p_raw.ICD_CODE.isin(icds)][['HADM_ID','ICD_CODE']].pivot_table(index='HADM_ID', columns='ICD_CODE', aggfunc='size', fill_value=np.nan)
p_raw = p_raw.reset_index()

p_raw.loc[:, p_raw.columns != 'HADM_ID'] = p_raw.loc[:, p_raw.columns != 'HADM_ID'].notna().astype(int)
print('p_raw_result',p_raw.shape,len(p_raw.HADM_ID.unique()))

p_raw = p_raw[p_raw.HADM_ID.isin(df.HADM_ID)]
print('p_raw_result',p_raw.shape,len(p_raw.HADM_ID.unique()))

final_p = df[['HADM_ID','ICUSTAY_ID']]
final_p = pd.merge(final_p,p_raw,on='HADM_ID',how='left')
print(final_p.shape,len(final_p.HADM_ID.unique()),len(final_p.ICUSTAY_ID.unique()))

final_p = final_p[~final_p[final_p.columns.difference(['HADM_ID', 'ICUSTAY_ID'])].isna().all(axis=1)]
final_p = final_p.fillna(0)
print(final_p.shape,len(final_p.HADM_ID.unique()),len(final_p.ICUSTAY_ID.unique()))

final_p.columns = final_p.columns.astype(str)
final_p.columns = ['Pro_' + col if (col != 'HADM_ID')&(col != 'ICUSTAY_ID') else col for col in final_p.columns]
final_p = final_p.loc[:, (final_p != 0).any(axis=0)]
final_p = final_p.drop('HADM_ID', axis=1)
final_p.head()

p_raw_result (170224, 112) 170224
p_raw_result (47420, 112) 47420
67225 67225) 
(47420, 113) 47420 47420


,ICUSTAY_ID,Pro_0014,Pro_0017,Pro_0040,Pro_0045,Pro_0066,Pro_009U3ZX,Pro_0159,Pro_02100Z9,Pro_02H633Z,...,Pro_9915,Pro_9920,Pro_9925,Pro_9955,Pro_9960,Pro_9962,Pro_9983,Pro_B2111ZZ,Pro_B211YZZ,Pro_B24BZZ4
0,37081114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,37067082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,37510196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,39060235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,34672098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
zero_cols = final_p.columns[final_p.eq(0).all()].tolist()
zero_cols

[]

In [173]:
final_p.shape

(47420, 111)

In [174]:
final_p.to_csv('D:/2025UTI/IV/IV_all_pro.csv',index=False)

In [176]:
iii_cols = ['ICUSTAY_ID', 'Pro_0014', 'Pro_0017', 'Pro_0040', 'Pro_0045',
       'Pro_0066', 'Pro_0159', 'Pro_0331', 'Pro_0741', 'Pro_311', 'Pro_3322',
       'Pro_3323', 'Pro_3324', 'Pro_3404', 'Pro_3491', 'Pro_3521', 'Pro_3522',
       'Pro_3606', 'Pro_3607', 'Pro_3611', 'Pro_3612', 'Pro_3613', 'Pro_3615',
       'Pro_3721', 'Pro_3722', 'Pro_3723', 'Pro_3761', 'Pro_3845', 'Pro_387',
       'Pro_3891', 'Pro_3893', 'Pro_3895', 'Pro_3897', 'Pro_3899', 'Pro_3950',
       'Pro_3961', 'Pro_3995', 'Pro_4311', 'Pro_4443', 'Pro_4513', 'Pro_4516',
       'Pro_4523', 'Pro_5185', 'Pro_5459', 'Pro_5491', 'Pro_7309', 'Pro_734',
       'Pro_7359', 'Pro_7569', 'Pro_8051', 'Pro_8162', 'Pro_8622', 'Pro_8659',
       'Pro_8744', 'Pro_8841', 'Pro_8842', 'Pro_8847', 'Pro_8848', 'Pro_8852',
       'Pro_8853', 'Pro_8855', 'Pro_8856', 'Pro_8872', 'Pro_8938', 'Pro_8964',
       'Pro_9390', 'Pro_9604', 'Pro_9605', 'Pro_9607', 'Pro_966', 'Pro_9671',
       'Pro_9672', 'Pro_9904', 'Pro_9905', 'Pro_9907', 'Pro_9910', 'Pro_9915',
       'Pro_9920', 'Pro_9925', 'Pro_9955', 'Pro_9960', 'Pro_9962']
len(iii_cols)

82

In [177]:
print(set(iii_cols)-set(final_p.columns))

set()


In [178]:
d_icd_procedures[d_icd_procedures.ICD_CODE.isin([i.split('_')[-1] for i in list(set(iii_cols)-set(final_p.columns))])]

,ICD_CODE,ICD_VERSION,ICD_TEXT


In [179]:
print(set(final_p.columns)-set(iii_cols))

{'Pro_5A1945Z', 'Pro_4A023N7', 'Pro_5A1221Z', 'Pro_3E0G76Z', 'Pro_10E0XZZ', 'Pro_3E04305', 'Pro_009U3ZX', 'Pro_9983', 'Pro_5A1D70Z', 'Pro_02HV33Z', 'Pro_3E0436Z', 'Pro_10907ZC', 'Pro_0BH17EZ', 'Pro_03HY32Z', 'Pro_8952', 'Pro_5A1935Z', 'Pro_0DJ08ZZ', 'Pro_0W9930Z', 'Pro_10D00Z1', 'Pro_0DJD8ZZ', 'Pro_B2111ZZ', 'Pro_0KQM0ZZ', 'Pro_B211YZZ', 'Pro_0W9G3ZZ', 'Pro_02H633Z', 'Pro_02100Z9', 'Pro_B24BZZ4', 'Pro_5A1955Z', 'Pro_0W9G3ZX'}


In [180]:
d_icd_procedures[d_icd_procedures.ICD_CODE.isin([i.split('_')[-1] for i in list(set(final_p.columns)-set(iii_cols))])].ICD_TEXT.values

array(['Repair Perineum Muscle, Open Approach',
       'Ultrasonography of Heart with Aorta, Transesophageal',
       'Fluoroscopy of Multiple Coronary Arteries using Other Contrast',
       'Fluoroscopy of Multiple Coronary Arteries using Low Osmolar Contrast',
       'Performance of Cardiac Output, Continuous',
       'Respiratory Ventilation, Less than 24 Consecutive Hours',
       'Performance of Urinary Filtration, Intermittent, Less than 6 Hours Per Day',
       'Respiratory Ventilation, Greater than 96 Consecutive Hours',
       'Respiratory Ventilation, 24-96 Consecutive Hours',
       'Delivery of Products of Conception, External Approach',
       'Introduction of Nutritional Substance into Upper GI, Via Natural or Artificial Opening',
       'Measurement of Cardiac Sampling and Pressure, Left Heart, Percutaneous Approach',
       'Introduction of Other Antineoplastic into Central Vein, Percutaneous Approach',
       'Introduction of Nutritional Substance into Central Vein, Pe

In [175]:
final_p.shape

(47420, 111)

In [182]:
final_p[iii_cols].to_csv('D:/2025UTI/IV/IV_all_pro_matchIII.csv',index=False)

# Med

In [211]:
PRESCRIPTIONS = pd.read_csv('D:/mimic-iv-3.1/mimiciv/3.1/hosp/prescriptions.csv.gz', dtype={'ndc':'category'})
PRESCRIPTIONS.columns = PRESCRIPTIONS.columns.str.upper()

PRESCRIPTIONS.rename(columns={'STAY_ID': 'ICUSTAY_ID','STARTTIME':'STARTDATE','STOPTIME':'ENDDATE'}, inplace=True)
print(PRESCRIPTIONS.shape)
PRESCRIPTIONS = PRESCRIPTIONS[PRESCRIPTIONS.HADM_ID.isin(df.HADM_ID)]
print(PRESCRIPTIONS.shape)

PRESCRIPTIONS.STARTDATE = pd.to_datetime(PRESCRIPTIONS.STARTDATE)
PRESCRIPTIONS.ENDDATE = pd.to_datetime(PRESCRIPTIONS.ENDDATE)

(20292611, 21)
(7432739, 21)


In [212]:
PRESCRIPTIONS['DRUG'] = PRESCRIPTIONS['DRUG'].str.strip().replace({

    'Azithromycin ': 'Azithromycin',
    'CeftriaXONE': 'Ceftriaxone',  # 统一为小写x
    'CefTRIAXone': 'Ceftriaxone',  # 补充可能存在的其他拼写
    'MetRONIDAZOLE (FLagyl)': 'Metronidazole',
    'Piperacillin-Tazobactam Na': 'Piperacillin-Tazobactam',
    'Piperacillin Tazobactam': 'Piperacillin-Tazobactam',
    'LevETIRAcetam': 'Levetiracetam',
    'CefTAZidime': 'Ceftazidime',
    'CefePIME': 'Cefepime',
    'FoLIC Acid': 'Folic Acid',
    'PredniSONE': 'Prednisone',
    
    'LORazepam': 'Lorazepam',
    'Midazolam HCl': 'Midazolam',
    'HYDROmorphone (Dilaudid)': 'Hydromorphone',
    'OxyCODONE (Immediate Release)': 'Oxycodone',
    'OxycoDONE (Immediate Release) ': 'Oxycodone',  # 建议去掉"(Immediate Release)"保持简洁
    'Oxycodone (Immediate Release)': 'Oxycodone',
    'LevETIRAcetam':'Levetiracetam',
    
    'Amiodarone HCl': 'Amiodarone',
    'Metoprolol Tartrate': 'Metoprolol',
    'Metoprolol Succinate XL': 'Metoprolol',
    'Phenylephrine HCl': 'Phenylephrine',
    'Hydralazine Hcl': 'Hydralazine',
    'Ciprofloxacin Hcl':'Ciprofloxacin',
    
    'Heparin Sodium': 'Heparin',
    'Heparin Flush (10 units/ml)': 'Heparin',
    'Heparin Dwell (1000 Units/mL)': 'Heparin',
    'Heparin Flush Cvl  (100 Units/Ml)': 'Heparin',
    
    'Potassium Chloride Replacement (Critical Care and Oncology)': 'Potassium Chloride',
    'Potassium Chloride (Powder)': 'Potassium Chloride',
    'Pantoprazole Sodium': 'Pantoprazole',
    'Aspirin EC': 'Aspirin',
    'Vancomycin HCl': 'Vancomycin',
    'Acetaminophen IV': 'Acetaminophen'
}).str.title()  # 统一为首字母大写格式

In [218]:
keep_drug = list(pd.unique(['Insulin', 'Furosemide','Magnesium Sulfate','Calcium Gluconate',
'Heparin', 'Metoprolol',
'Acetaminophen',
'Hydromorphone','Fentanyl Citrate','Morphine Sulfate','Oxycodone',
'Propofol','Midazolam','Lorazepam','Dexmedetomidine',
'Aspirin',
'Pantoprazole','Famotidine','Ranitidine','Omeprazole',
'Ondansetron','Metoclopramide',
'Amiodarone','Albumin 25% (12.5g / 50mL)',
'Potassium Chloride',
'Levetiracetam','Gabapentin',
'Meropenem',
'Prednisone','Dexamethasone',
'Tacrolimus','Diltiazem','Thiamine','Folic Acid','Sodium Bicarbonate',
'Vancomycin', #万古霉素 - 用于MRSA（耐甲氧西林金黄色葡萄球菌）感染
'Piperacillin-Tazobactam', #哌拉西林-他唑巴坦 - 广谱抗生素
'Meropenem', #美罗培南 - 用于多重耐药革兰阴性菌（如铜绿假单胞菌）感染
'Cefepime', #头孢吡肟 - 覆盖革兰阴性菌（包括假单胞菌）
'Ceftazidime', #针对铜绿假单胞菌等耐药革兰阴性菌，可能反映耐药菌感染或免疫功能低下状态
'Ampicillin-Sulbactam', #氨苄西林-舒巴坦 - 阳性肠杆菌科细菌
'Linezolid', #用于耐万古霉素肠球菌（VRE）或其他耐药革兰阳性菌
'Azithromycin', #阿奇霉素 - 治疗 细菌感染 和某些 非典型病原体感染
'Ceftriaxone', #获得性肺炎或脑膜炎
'Clindamycin',
'Phenylephrine','Norepinephrine','Nitroglycerin', 'Metronidazole',
'Levofloxacin', 'Ciprofloxacin', 'Haloperidol','Simvastatin',
'Atorvastatin','Glucagon','Warfarin','Lactulose']))
len(keep_drug)

56

In [219]:
print(keep_drug)

['Insulin', 'Furosemide', 'Magnesium Sulfate', 'Calcium Gluconate', 'Heparin', 'Metoprolol', 'Acetaminophen', 'Hydromorphone', 'Fentanyl Citrate', 'Morphine Sulfate', 'Oxycodone', 'Propofol', 'Midazolam', 'Lorazepam', 'Dexmedetomidine', 'Aspirin', 'Pantoprazole', 'Famotidine', 'Ranitidine', 'Omeprazole', 'Ondansetron', 'Metoclopramide', 'Amiodarone', 'Albumin 25% (12.5g / 50mL)', 'Potassium Chloride', 'Levetiracetam', 'Gabapentin', 'Meropenem', 'Prednisone', 'Dexamethasone', 'Tacrolimus', 'Diltiazem', 'Thiamine', 'Folic Acid', 'Sodium Bicarbonate', 'Vancomycin', 'Piperacillin-Tazobactam', 'Cefepime', 'Ceftazidime', 'Ampicillin-Sulbactam', 'Linezolid', 'Azithromycin', 'Ceftriaxone', 'Clindamycin', 'Phenylephrine', 'Norepinephrine', 'Nitroglycerin', 'Metronidazole', 'Levofloxacin', 'Ciprofloxacin', 'Haloperidol', 'Simvastatin', 'Atorvastatin', 'Glucagon', 'Warfarin', 'Lactulose']


In [220]:
PRESCRIPTIONS = PRESCRIPTIONS[PRESCRIPTIONS.DRUG.isin(keep_drug)]
print(PRESCRIPTIONS.shape,len(PRESCRIPTIONS.DRUG.unique()))

(3306944, 21) 55


In [221]:
iii_m = ['Tacrolimus', 'Warfarin', 'Heparin', 'Furosemide', 'Nitroglycerin', 'Insulin', 'Midazolam', 'Lorazepam', 'Magnesium Sulfate', 'Potassium Chloride', 'Vancomycin', 'Acetaminophen', 'Metoclopramide', 'Morphine Sulfate', 'Calcium Gluconate', 'Pantoprazole', 'Levofloxacin', 'Prednisone', 'Clindamycin', 'Ranitidine', 'Hydromorphone', 'Metoprolol', 'Metronidazole', 'Fentanyl Citrate', 'Haloperidol', 'Piperacillin-Tazobactam', 'Atorvastatin', 'Aspirin', 'Amiodarone', 'Meropenem', 'Phenylephrine', 'Sodium Bicarbonate', 'Propofol', 'Norepinephrine', 'Dexamethasone', 'Ondansetron', 'Levetiracetam', 'Ceftazidime', 'Folic Acid', 'Famotidine', 'Simvastatin', 'Linezolid', 'Ceftriaxone', 'Lactulose', 'Omeprazole', 'Diltiazem', 'Cefepime', 'Oxycodone', 'Azithromycin', 'Gabapentin', 'Ampicillin-Sulbactam', 'Ciprofloxacin', 'Glucagon', 'Dexmedetomidine', 'Thiamine']

In [222]:
print(set(PRESCRIPTIONS.DRUG.unique())-set(iii_m))

set()


In [224]:
# 将一些写了逗号，的数字修改
PRESCRIPTIONS['DOSE_VAL_RX'] = PRESCRIPTIONS['DOSE_VAL_RX'].astype(str).str.replace(',', '', regex=False)
print(PRESCRIPTIONS.shape, PRESCRIPTIONS.shape)

(3306944, 21) (3306944, 21)


In [232]:
PRESCRIPTIONS['DOSE_UNIT_RX'] = PRESCRIPTIONS['DOSE_UNIT_RX'].replace({
    # 质量单位标准化
    'gm': 'g',
    'mg/ml': 'mg/mL',
    'mg/ml': 'mg/mL',
    'mcg/ml': 'mcg/mL',
    'mg/50 ml': 'mg/mL',
    'mg/250 ml': 'mg/mL',
    'mg/m2': 'mg',
    'mg/kg/hr': 'mg',
    'mg/hr': 'mg',
    'mg/min': 'mg',
    'mg/dose': 'mg',
    'mg\\ 0 mg': 'mg',

    # 容积单位标准化
    'ml': 'mL',
    'L': 'L',

    # 单位统一
    'Units': 'UNIT',
    'units': 'UNIT',
    '_UNIT': 'UNIT',
    'Units/Liter': 'UNIT',
    'units/hr': 'UNIT',
    'UNIT/HR': 'UNIT',

    # 剂量形式
    'MD to order daily dose': 'dose',
    'Dosing by Pharmacy': 'dose',
    'dose': 'dose',
    'TAB': 'tab',
    'CAP': 'cap',
    'SYR': 'syr',
    'DROP': 'drop',
    'VIAL': 'vial',
    'AMP': 'amp',
    'INJ': 'inj',
    'Appl': 'appl',

    # mcg 相关标准化
    'mcg/kg/min': 'mcg',
    'mcg/kg/hr': 'mcg',
    'mcg/hr': 'mcg',

    # 特殊无效项处理
    'Pick Option Below': None,
    'in': None,  # 非法单位，丢弃
    np.nan: None,
})


In [233]:
PRESCRIPTIONS['DOSE_UNIT_RX'].unique()

array(['mg', 'g', 'mL', 'UNIT', 'mEq', 'mcg', 'dose', None, 'cap', 'appl',
       'tab', 'mg/mL', 'mmol', 'amp', 'drop', '%', 'mcg/mL', 'syr'],
      dtype=object)

In [234]:
PRESCRIPTIONS.DOSE_UNIT_RX.value_counts()

mg        1896939
UNIT       492400
mEq        409664
g          398643
mL          64718
mcg         31488
dose        11277
cap            26
appl           24
%              11
drop            6
tab             5
mg/mL           3
mmol            3
amp             2
mcg/mL          2
syr             1
Name: DOSE_UNIT_RX, dtype: int64

In [235]:
def process_dose_values(df, column='DOSE_VAL_RX'):
    """
    处理 DOSE_VAL_RX 列中的范围剂量，将形如 '325-650' 的字符串替换为其平均值。
    """
    def parse_dose(value):
        if isinstance(value, str):
            # 替换所有非标准破折号为标准的减号 "-"
            value = value.replace('–', '-').replace('—', '-').replace('－', '-').strip()
            # 正则匹配整数或小数范围
            match = re.match(r'^(\d*\.?\d+)\s*-\s*(\d*\.?\d+)$', value)
            if match:
                low = float(match.group(1))
                high = float(match.group(2))
                return (low + high) / 2
        # 尝试将其他格式直接转为 float
        try:
            return float(value)
        except:
            return np.nan

    df[column + '_processed'] = df[column].apply(parse_dose)
    return df

In [236]:
PRESCRIPTIONS = process_dose_values(PRESCRIPTIONS)

In [237]:
PRESCRIPTIONS.head(10)[['DOSE_VAL_RX','DOSE_VAL_RX_processed']].head()

,DOSE_VAL_RX,DOSE_VAL_RX_processed
222,12.5,12.5
224,1000,1000.0
226,20,20.0
227,0.5,0.5
229,60,60.0


In [238]:
# 转换剂量数值为统一单位，注意，gm 指的是 g
def unify_units(df):
    def convert_group(group):
        if 'DOSE_UNIT_RX' not in group.columns or 'DOSE_VAL_RX_processed' not in group.columns:
            return group
        
        # 获取出现频率最高的单位作为目标单位
        unit_counts = group['DOSE_UNIT_RX'].value_counts(dropna=True)
        if unit_counts.empty:
            return group
        
        target_unit = unit_counts.idxmax()

        # 单位换算关系：统一到目标单位
        conversion_factors = {
            ('mg', 'g'): 1 / 1000,
            ('g', 'mg'): 1000,
            ('mcg', 'mg'): 1 / 1000,
            ('mg', 'mcg'): 1000,
            ('mcg', 'g'): 1 / 1_000_000,
            ('g', 'mcg'): 1_000_000,
        }

        for from_unit in group['DOSE_UNIT_RX'].dropna().unique():
            if from_unit == target_unit:
                continue
            key = (from_unit, target_unit)
            if key in conversion_factors:
                factor = conversion_factors[key]
                mask = group['DOSE_UNIT_RX'] == from_unit
                group.loc[mask, 'DOSE_VAL_RX_processed'] = (
                    group.loc[mask, 'DOSE_VAL_RX_processed'].astype(float) * factor
                )
                group.loc[mask, 'DOSE_UNIT_RX'] = target_unit
            else:
                print(f"⚠️ 无法将单位从 {from_unit} 转换为 {target_unit}，保留原单位")

        return group

    # 防止 SettingWithCopyWarning
    df = df.groupby('DRUG', group_keys=False).apply(lambda x: convert_group(x.copy()))
    return df


In [239]:
PRESCRIPTIONS = unify_units(PRESCRIPTIONS)

⚠️ 无法将单位从 cap 转换为 mg，保留原单位
⚠️ 无法将单位从 drop 转换为 mg，保留原单位
⚠️ 无法将单位从 drop 转换为 mg，保留原单位
⚠️ 无法将单位从 appl 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 tab 转换为 mg，保留原单位
⚠️ 无法将单位从 % 转换为 mg，保留原单位
⚠️ 无法将单位从 tab 转换为 mg，保留原单位
⚠️ 无法将单位从 dose 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 UNIT，保留原单位
⚠️ 无法将单位从 mg 转换为 UNIT，保留原单位
⚠️ 无法将单位从 mcg/mL 转换为 mg，保留原单位
⚠️ 无法将单位从 g 转换为 mL，保留原单位
⚠️ 无法将单位从 mg/mL 转换为 mg，保留原单位
⚠️ 无法将单位从 drop 转换为 mg，保留原单位
⚠️ 无法将单位从 mEq 转换为 g，保留原单位
⚠️ 无法将单位从 mg/mL 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 appl 转换为 mg，保留原单位
⚠️ 无法将单位从 mg/mL 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 tab 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 mL 转换为 mg，保留原单位
⚠️ 无法将单位从 mg 转换为 mEq，保留原单位
⚠️ 无法将单位从 mmol 转换为 mEq，保留原单位
⚠️ 无法将单位从 mL 转换为 mEq，保留原单位
⚠️ 无法将单位从 dose 转换为 mg，保留原单位
⚠️ 无法将单位从 cap 转换为 mg，保留原单位
⚠️ 无法将单位从 mg 转换为 mEq，保留原单位
⚠️ 无法将单位从 tab 转换为 mEq，保留原单位
⚠️ 无法将单位从 amp 转换为 mEq，保留原单位
⚠️ 无法将单位从 mL 转换为 mEq，保留原单位
⚠️ 无法将单位从 syr 转换为 mEq，保留原单位
⚠️ 无法将单位从 dose 转换为 mg，保留原单位
⚠️ 无法将单位从 tab 转

In [240]:
for i in keep_drug:
    print(i,'\n',PRESCRIPTIONS[PRESCRIPTIONS.DRUG == i].DOSE_UNIT_RX.value_counts(),'\n')

Insulin 
 UNIT    304911
Name: DOSE_UNIT_RX, dtype: int64 

Furosemide 
 mg    222941
Name: DOSE_UNIT_RX, dtype: int64 

Magnesium Sulfate 
 g      177887
mEq        27
Name: DOSE_UNIT_RX, dtype: int64 

Calcium Gluconate 
 g    116728
Name: DOSE_UNIT_RX, dtype: int64 

Heparin 
 UNIT    187489
mL       41569
mg           3
Name: DOSE_UNIT_RX, dtype: int64 

Metoprolol 
 mg    177862
mL         2
Name: DOSE_UNIT_RX, dtype: int64 

Acetaminophen 
 mg    185515
Name: DOSE_UNIT_RX, dtype: int64 

Hydromorphone 
 mg        92787
mcg/mL        2
Name: DOSE_UNIT_RX, dtype: int64 

Fentanyl Citrate 
 mcg    55410
Name: DOSE_UNIT_RX, dtype: int64 

Morphine Sulfate 
 mg    49777
Name: DOSE_UNIT_RX, dtype: int64 

Oxycodone 
 mg     43265
mL         1
tab        1
Name: DOSE_UNIT_RX, dtype: int64 

Propofol 
 mg    60244
Name: DOSE_UNIT_RX, dtype: int64 

Midazolam 
 mg    24516
Name: DOSE_UNIT_RX, dtype: int64 

Lorazepam 
 mg    72502
Name: DOSE_UNIT_RX, dtype: int64 

Dexmedetomidine 
 mg   

In [241]:
PRESCRIPTIONS_keep_f = pd.merge(PRESCRIPTIONS[['HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG',
       'GSN', 'NDC', 'DOSE_VAL_RX_processed']],df[['HADM_ID','ICUSTAY_ID','ADMITTIME','DISCHTIME','INTIME','OUTTIME']],on='HADM_ID',how='left')

In [244]:
df.shape

(67225, 27)

In [243]:
PRESCRIPTIONS_keep_f.head(2)

,HADM_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,GSN,NDC,DOSE_VAL_RX_processed,ICUSTAY_ID,ADMITTIME,DISCHTIME,INTIME,OUTTIME
0,25860671,2150-11-07 20:00:00,2150-11-12 18:00:00,MAIN,Metoprolol,050631,51079025520,12.5,37081114,2150-11-02 18:02:00,2150-11-12 13:45:00,2150-11-02 19:37:00,2150-11-06 17:03:17
1,25860671,2150-11-09 08:00:00,2150-11-08 10:00:00,MAIN,Vancomycin,009331,00409433201,1000.0,37081114,2150-11-02 18:02:00,2150-11-12 13:45:00,2150-11-02 19:37:00,2150-11-06 17:03:17


In [245]:
PRESCRIPTIONS_keep_f.shape

(3306944, 13)

In [246]:
PRESCRIPTIONS_keep_f.to_csv('D:/2025UTI/IV/IV_all_med.csv',index=False)

In [247]:
PRESCRIPTIONS_keep_f.columns

Index(['HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG', 'GSN', 'NDC',
       'DOSE_VAL_RX_processed', 'ICUSTAY_ID', 'ADMITTIME', 'DISCHTIME',
       'INTIME', 'OUTTIME'],
      dtype='object')

In [248]:
date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','STARTDATE', 'ENDDATE']
PRESCRIPTIONS_keep_f[date_columns] = PRESCRIPTIONS_keep_f[date_columns].apply(pd.to_datetime)

In [249]:
# 先用maks保留 药物使用在入院至ICU结束期间的
mask = (PRESCRIPTIONS_keep_f['STARTDATE'] >= PRESCRIPTIONS_keep_f['ADMITTIME']) & (PRESCRIPTIONS_keep_f['ENDDATE'] <= PRESCRIPTIONS_keep_f['OUTTIME'])
PRESCRIPTIONS_keep_f = PRESCRIPTIONS_keep_f[mask]
PRESCRIPTIONS_keep_f.shape

(1589155, 13)

In [250]:
PRESCRIPTIONS_keep_f['DRUG_hr_inICu'] = PRESCRIPTIONS_keep_f['STARTDATE'] - PRESCRIPTIONS_keep_f['INTIME']
PRESCRIPTIONS_keep_f['DRUG_hr_inICu'] = PRESCRIPTIONS_keep_f['DRUG_hr_inICu'].dt.total_seconds() / 3600

In [251]:
PRESCRIPTIONS_keep_f.head(2)

,HADM_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,GSN,NDC,DOSE_VAL_RX_processed,ICUSTAY_ID,ADMITTIME,DISCHTIME,INTIME,OUTTIME,DRUG_hr_inICu
3,25860671,2150-11-03 22:00:00,2150-11-04 21:00:00,MAIN,Haloperidol,003970,55390014710,0.5,37081114,2150-11-02 18:02:00,2150-11-12 13:45:00,2150-11-02 19:37:00,2150-11-06 17:03:17,26.383333
4,25860671,2150-11-03 00:00:00,2150-11-06 12:00:00,MAIN,Phenylephrine,005068,10019016312,60.0,37081114,2150-11-02 18:02:00,2150-11-12 13:45:00,2150-11-02 19:37:00,2150-11-06 17:03:17,4.383333


In [252]:
# 从住院到进入ICU的 48小时内 的药物input sum
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f[PRESCRIPTIONS_keep_f.DRUG_hr_inICu<=48]
print(PRESCRIPTIONS_keep_f_48.shape, len(PRESCRIPTIONS_keep_f_48.ICUSTAY_ID.unique()))
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f_48.groupby(['ICUSTAY_ID', 'DRUG'])['DOSE_VAL_RX_processed'].sum().unstack(fill_value=0).reset_index()
PRESCRIPTIONS_keep_f_48.head()

(1113709, 14) 65034


DRUG,ICUSTAY_ID,Acetaminophen,Amiodarone,Ampicillin-Sulbactam,Aspirin,Atorvastatin,Azithromycin,Calcium Gluconate,Cefepime,Ceftazidime,...,Potassium Chloride,Prednisone,Propofol,Ranitidine,Simvastatin,Sodium Bicarbonate,Tacrolimus,Thiamine,Vancomycin,Warfarin
0,30000153,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,1200.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
1,30000213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0
3,30000646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0
4,30000831,1000.0,0.0,0.0,381.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [253]:
PRESCRIPTIONS_keep_f_48.shape

(65034, 56)

In [257]:
zero_columns = PRESCRIPTIONS_keep_f_48.columns[(PRESCRIPTIONS_keep_f_48 == 0).all()]
print("全部为0的列有：", list(zero_columns))

全部为0的列有： []


In [260]:
PRESCRIPTIONS_keep_f_48 = PRESCRIPTIONS_keep_f_48.rename(columns={col: f"Med_{col}" for col in PRESCRIPTIONS_keep_f_48.columns if col != 'ICUSTAY_ID'})

In [261]:
PRESCRIPTIONS_keep_f_48.columns

Index(['ICUSTAY_ID', 'Med_Acetaminophen', 'Med_Amiodarone',
       'Med_Ampicillin-Sulbactam', 'Med_Aspirin', 'Med_Atorvastatin',
       'Med_Azithromycin', 'Med_Calcium Gluconate', 'Med_Cefepime',
       'Med_Ceftazidime', 'Med_Ceftriaxone', 'Med_Ciprofloxacin',
       'Med_Clindamycin', 'Med_Dexamethasone', 'Med_Dexmedetomidine',
       'Med_Diltiazem', 'Med_Famotidine', 'Med_Fentanyl Citrate',
       'Med_Folic Acid', 'Med_Furosemide', 'Med_Gabapentin', 'Med_Glucagon',
       'Med_Haloperidol', 'Med_Heparin', 'Med_Hydromorphone', 'Med_Insulin',
       'Med_Lactulose', 'Med_Levetiracetam', 'Med_Levofloxacin',
       'Med_Linezolid', 'Med_Lorazepam', 'Med_Magnesium Sulfate',
       'Med_Meropenem', 'Med_Metoclopramide', 'Med_Metoprolol',
       'Med_Metronidazole', 'Med_Midazolam', 'Med_Morphine Sulfate',
       'Med_Nitroglycerin', 'Med_Norepinephrine', 'Med_Omeprazole',
       'Med_Ondansetron', 'Med_Oxycodone', 'Med_Pantoprazole',
       'Med_Phenylephrine', 'Med_Piperacillin-Tazob

In [262]:
nonzero_ratio = (PRESCRIPTIONS_keep_f_48 != 0).sum() / len(PRESCRIPTIONS_keep_f_48)
print(nonzero_ratio)

DRUG
ICUSTAY_ID                     1.000000
Med_Acetaminophen              0.462004
Med_Amiodarone                 0.060169
Med_Ampicillin-Sulbactam       0.018513
Med_Aspirin                    0.232970
Med_Atorvastatin               0.102454
Med_Azithromycin               0.067319
Med_Calcium Gluconate          0.318941
Med_Cefepime                   0.146493
Med_Ceftazidime                0.022342
Med_Ceftriaxone                0.113879
Med_Ciprofloxacin              0.000215
Med_Clindamycin                0.015792
Med_Dexamethasone              0.052250
Med_Dexmedetomidine            0.093059
Med_Diltiazem                  0.041517
Med_Famotidine                 0.193284
Med_Fentanyl Citrate           0.311591
Med_Folic Acid                 0.049774
Med_Furosemide                 0.311975
Med_Gabapentin                 0.046560
Med_Glucagon                   0.138897
Med_Haloperidol                0.048221
Med_Heparin                    0.455208
Med_Hydromorphone              0.20

In [264]:
PRESCRIPTIONS_keep_f_48.shape

(65034, 56)

In [263]:
PRESCRIPTIONS_keep_f_48.to_csv('D:/2025UTI/IV/IV_all_med_matchIII_48.csv',index=False)

# Static Concat

In [2]:
df = pd.read_csv('D:/2025UTI/IV/MIMIC4_p.csv')
print('------------------------------------------------------------------------')
print('Final Data：',df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
print('DIEINICU:',Counter(df['DIEINICU']))
print('Readmission_30:',Counter(df['Readmission_30']))
print('Readmission_60:',Counter(df['Readmission_60']))
print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

------------------------------------------------------------------------
Final Data： (67225, 27) SUBJECT_ID: 53715 HADM_ID: 67225 ICUSTAY_ID: 67225
DIEINHOSPITAL: Counter({0: 59635, 1: 7590})
DIEINICU: Counter({0: 62220, 1: 5005})
Readmission_30: Counter({0: 64796, 1: 2429})
Readmission_60: Counter({0: 62993, 1: 4232})
ICU_within_12hr_of_admit: Counter({1: 47012, 0: 20213})
Multiple_ICUs: Counter({0: 61177, 1: 6048})


In [3]:
diag = pd.read_csv('D:/2025UTI/IV/IV_all_diag_matchIII.csv')
proc = pd.read_csv('D:/2025UTI/IV/IV_all_pro_matchIII.csv')
med = pd.read_csv('D:/2025UTI/IV/IV_all_med_matchIII_48.csv')

In [5]:
print(diag.shape, proc.shape, med.shape)

(66981, 211) (47420, 82) (65034, 56)


In [7]:
diag.columns

Index(['HADM_ID', 'Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570',
       'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464',
       'Diag_9_7470',
       ...
       'Group_CKD', 'Group_PCOS', 'Group_Neoplasm_ovary',
       'Group_Endometriosis', 'Group_Leiomyoma', 'Group_Va_uti',
       'Diag_Personal history of antineoplastic chemotherapy',
       'Diag_Personal history of pulmonary embolism',
       'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits',
       'Diag_Pneumonia, unspecified organism'],
      dtype='object', length=211)

In [8]:
proc.columns

Index(['ICUSTAY_ID', 'Pro_0014', 'Pro_0017', 'Pro_0040', 'Pro_0045',
       'Pro_0066', 'Pro_0159', 'Pro_0331', 'Pro_0741', 'Pro_311', 'Pro_3322',
       'Pro_3323', 'Pro_3324', 'Pro_3404', 'Pro_3491', 'Pro_3521', 'Pro_3522',
       'Pro_3606', 'Pro_3607', 'Pro_3611', 'Pro_3612', 'Pro_3613', 'Pro_3615',
       'Pro_3721', 'Pro_3722', 'Pro_3723', 'Pro_3761', 'Pro_3845', 'Pro_387',
       'Pro_3891', 'Pro_3893', 'Pro_3895', 'Pro_3897', 'Pro_3899', 'Pro_3950',
       'Pro_3961', 'Pro_3995', 'Pro_4311', 'Pro_4443', 'Pro_4513', 'Pro_4516',
       'Pro_4523', 'Pro_5185', 'Pro_5459', 'Pro_5491', 'Pro_7309', 'Pro_734',
       'Pro_7359', 'Pro_7569', 'Pro_8051', 'Pro_8162', 'Pro_8622', 'Pro_8659',
       'Pro_8744', 'Pro_8841', 'Pro_8842', 'Pro_8847', 'Pro_8848', 'Pro_8852',
       'Pro_8853', 'Pro_8855', 'Pro_8856', 'Pro_8872', 'Pro_8938', 'Pro_8964',
       'Pro_9390', 'Pro_9604', 'Pro_9605', 'Pro_9607', 'Pro_966', 'Pro_9671',
       'Pro_9672', 'Pro_9904', 'Pro_9905', 'Pro_9907', 'Pro_9910',

In [9]:
med.columns

Index(['ICUSTAY_ID', 'Med_Acetaminophen', 'Med_Amiodarone',
       'Med_Ampicillin-Sulbactam', 'Med_Aspirin', 'Med_Atorvastatin',
       'Med_Azithromycin', 'Med_Calcium Gluconate', 'Med_Cefepime',
       'Med_Ceftazidime', 'Med_Ceftriaxone', 'Med_Ciprofloxacin',
       'Med_Clindamycin', 'Med_Dexamethasone', 'Med_Dexmedetomidine',
       'Med_Diltiazem', 'Med_Famotidine', 'Med_Fentanyl Citrate',
       'Med_Folic Acid', 'Med_Furosemide', 'Med_Gabapentin', 'Med_Glucagon',
       'Med_Haloperidol', 'Med_Heparin', 'Med_Hydromorphone', 'Med_Insulin',
       'Med_Lactulose', 'Med_Levetiracetam', 'Med_Levofloxacin',
       'Med_Linezolid', 'Med_Lorazepam', 'Med_Magnesium Sulfate',
       'Med_Meropenem', 'Med_Metoclopramide', 'Med_Metoprolol',
       'Med_Metronidazole', 'Med_Midazolam', 'Med_Morphine Sulfate',
       'Med_Nitroglycerin', 'Med_Norepinephrine', 'Med_Omeprazole',
       'Med_Ondansetron', 'Med_Oxycodone', 'Med_Pantoprazole',
       'Med_Phenylephrine', 'Med_Piperacillin-Tazob

In [10]:
Final_df = pd.merge(df,diag,on='HADM_ID',how='left')
Final_df = pd.merge(Final_df,proc,on='ICUSTAY_ID',how='left')
Final_df = pd.merge(Final_df,med,on='ICUSTAY_ID',how='left')
print(df.shape, Final_df.shape)

(67225, 27) (67225, 373)


In [12]:
dpm_cols = list(pd.unique(diag.columns.tolist() + proc.columns.tolist() + med.columns.tolist()))
dpm_cols = list(set(dpm_cols)-set(['HADM_ID','ICUSTAY_ID']))
len(dpm_cols)

346

In [16]:
iii_acols = ['Pro_9607', 'Diag_9_E8798', 'Diag_9_66411', 'Diag_10_K22', 'Diag_10_R00', 'Pro_3722', 'Diag_10_C33', 'Pro_9905', 'Pro_8841', 'Diag_10_I73', 'Pro_3606', 'Pro_8842', 'Diag_10_I47', 'Diag_9_V420', 'Diag_9_86121', 'Pro_0017', 'Diag_9_99731', 'Diag_10_R09', 'Med_Piperacillin-Tazobactam', 'Pro_3895', 'Pro_8852', 'Diag_10_I67', 'Diag_9_43310', 'Group_Va_uti', 'Diag_9_6930', 'Diag_10_I82', 'Diag_10_R06', 'Med_Potassium Chloride', 'Diag_10_G97', 'Pro_3615', 'Pro_0040', 'Pro_9907', 'Med_Diltiazem', 'Pro_0014', 'Med_Ciprofloxacin', 'Diag_10_B95', 'Diag_10_C61', 'Pro_734', 'Med_Morphine Sulfate', 'Med_Vancomycin', 'Diag_10_I34', 'Diag_10_Z80', 'Diag_10_B37', 'Diag_10_L89', 'Pro_9925', 'Pro_9960', 'Diag_10_D69', 'Diag_9_E9320', 'Pro_8847', 'Med_Famotidine', 'Diag_10_I25', 'Med_Oxycodone', 'Diag_10_Z91', 'Pro_9955', 'Med_Metoclopramide', 'Diag_10_J93', 'Diag_10_E44', 'Med_Tacrolimus', 'Pro_9604', 'Diag_10_E78', 'Diag_10_G20', 'Med_Thiamine', 'Diag_10_D61', 'Med_Levofloxacin', 'Pro_8964', 'Pro_3322', 'Diag_10_I60', 'Pro_9671', 'Diag_10_S01', 'Diag_10_F05', 'Diag_9_9975', 'Pro_3721', 'Diag_10_I31', 'Diag_10_Z23', 'Pro_8938', 'Group_Leiomyoma', 'Diag_9_V462', 'Med_Ranitidine', 'Diag_10_C22', 'Pro_3491', 'Diag_10_R63', 'Pro_3404', 'Diag_10_K70', 'Diag_10_E22', 'Diag_10_J41', 'Med_Prednisone', 'Pro_7359', 'Diag_10_J44', 'Pro_8853', 'Diag_Personal history of antineoplastic chemotherapy', 'Diag_10_G40', 'Diag_10_D66', 'Pro_3613', 'Diag_9_9974', 'Diag_10_K80', 'Diag_10_G89', 'Med_Phenylephrine', 'Med_Ceftazidime', 'Diag_10_S12', 'Pro_387', 'Diag_10_C78', 'Diag_10_G61', 'Med_Furosemide', 'Diag_10_N23', 'Diag_10_E28', 'Diag_10_F30', 'Group_AKI', 'Med_Simvastatin', 'Med_Haloperidol', 'Med_Folic Acid', 'Med_Metronidazole', 'Diag_9_V1251', 'Diag_10_I26', 'Med_Midazolam', 'Pro_0045', 'Med_Magnesium Sulfate', 'Diag_10_N72', 'Med_Atorvastatin', 'Diag_10_C82', 'Med_Lactulose', 'Diag_10_E87', 'Diag_10_I50', 'Diag_10_M81', 'Diag_10_G30', 'Diag_10_R69', 'Diag_10_J43', 'Diag_10_Z87', 'Diag_10_K56', 'Diag_9_V422', 'Diag_10_K92', 'Diag_9_7464', 'Diag_10_F04', 'Diag_10_T78', 'Med_Omeprazole', 'Med_Calcium Gluconate', 'Med_Norepinephrine', 'Med_Ondansetron', 'Diag_10_D50', 'Group_Endometriosis', 'Diag_10_K76', 'Diag_9_V433', 'Pro_3521', 'Diag_10_I61', 'Diag_10_I66', 'Diag_10_C79', 'Pro_4523', 'Med_Dexmedetomidine', 'Diag_10_R10', 'Group_PCOS', 'Diag_10_I20', 'Diag_10_Z95', 'Med_Warfarin', 'Diag_10_I08', 'Diag_9_2113', 'Diag_10_B15', 'Diag_10_N05', 'Diag_10_N13', 'Pro_3611', 'Pro_8744', 'Pro_311', 'Diag_10_R78', 'Pro_9915', 'Diag_10_A40', 'Diag_10_M15', 'Diag_10_G93', 'Diag_10_R58', 'Diag_10_N28', 'Diag_10_K31', 'Diag_10_M06', 'Pro_3950', 'Diag_10_I12', 'Diag_10_F10', 'Med_Sodium Bicarbonate', 'Group_Neoplasm_ovary', 'Med_Nitroglycerin', 'Med_Azithromycin', 'Diag_10_H40', 'Diag_Pneumonia, unspecified organism', 'Diag_10_K66', 'Pro_3522', 'Pro_8162', 'Diag_10_E11', 'Diag_9_E9478', 'Med_Ceftriaxone', 'Diag_10_I62', 'Diag_10_I70', 'Diag_10_G44', 'Diag_10_I33', 'Diag_10_R68', 'Diag_9_7470', 'Diag_10_S06', 'Group_CKD', 'Diag_9_99702', 'Pro_4516', 'Med_Acetaminophen', 'Diag_9_V4365', 'Med_Gabapentin', 'Diag_10_I95', 'Diag_10_H35', 'Diag_10_N39', 'Med_Amiodarone', 'Pro_3324', 'Pro_3612', 'Diag_10_M79', 'Diag_10_F19', 'Pro_3845', 'Diag_10_Z00', 'Diag_9_E9342', 'Pro_4513', 'Pro_5459', 'Diag_10_Z81', 'Pro_9672', 'Diag_10_K85', 'Diag_10_I85', 'Med_Pantoprazole', 'Diag_10_Q20', 'Diag_10_E43', 'Diag_10_I07', 'Pro_3995', 'Diag_10_Z85', 'Pro_3323', 'Diag_9_E8788', 'Diag_9_39891', 'Diag_9_V1259', 'Diag_9_E8786', 'Diag_10_K91', 'Diag_10_Z16', 'Diag_10_A04', 'Diag_10_J84', 'Pro_9390', 'Diag_9_E8780', 'Diag_9_E8497', 'Diag_9_E8781', 'Diag_10_Z51', 'Diag_10_J15', 'Pro_7569', 'Diag_9_E8499', 'Diag_10_I10', 'Diag_10_E89', 'Diag_10_N92', 'Diag_10_F41', 'Pro_9920', 'Diag_10_R19', 'Diag_9_9973', 'Med_Dexamethasone', 'Diag_10_B20', 'Diag_10_K75', 'Med_Insulin', 'Diag_9_64891', 'Med_Lorazepam', 'Pro_4443', 'Pro_3893', 'Pro_9605', 'Diag_10_D70', 'Pro_8051', 'Med_Propofol', 'Diag_10_J45', 'Pro_3723', 'Diag_10_I42', 'Diag_10_O75', 'Pro_8848', 'Diag_9_V440', 'Diag_10_K41', 'Diag_10_F43', 'Diag_10_M60', 'Diag_10_J69', 'Med_Meropenem', 'Diag_9_E9331', 'Diag_10_I24', 'Diag_10_E66', 'Med_Fentanyl Citrate', 'Diag_10_M48', 'Diag_10_T81', 'Diag_10_G43', 'Diag_10_G35', 'Diag_10_G60', 'Pro_8856', 'Med_Aspirin', 'Pro_966', 'Diag_9_V6284', 'Med_Levetiracetam', 'Diag_9_E8490', 'Med_Cefepime', 'Diag_10_T82', 'Pro_3607', 'Diag_10_E83', 'Med_Heparin', 'Med_Linezolid', 'Diag_10_G81', 'Pro_3761', 'Diag_9_V441', 'Pro_3961', 'Diag_9_E8498', 'Diag_9_E8782', 'Diag_10_I69', 'Diag_10_R45', 'Pro_3899', 'Diag_9_5570', 'Pro_9910', 'Pro_3891', 'Pro_9962', 'Diag_Personal history of pulmonary embolism', 'Pro_3897', 'Med_Clindamycin', 'Pro_5491', 'Diag_10_I44', 'Pro_5185', 'Pro_0066', 'Diag_10_Z21', 'Pro_9904', 'Diag_10_K57', 'Pro_4311', 'Diag_10_K59', 'Pro_8622', 'Diag_10_M32', 'Diag_10_F32', 'Pro_0159', 'Med_Ampicillin-Sulbactam', 'Pro_7309', 'Med_Hydromorphone', 'Pro_8855', 'Med_Metoprolol', 'Diag_10_I21', 'Diag_10_M10', 'Diag_10_I71', 'Diag_10_J95', 'Med_Glucagon', 'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits', 'Diag_9_E8859', 'Pro_8659', 'Diag_9_E8790', 'Pro_8872', 'Diag_10_G47', 'Diag_10_D64', 'Diag_10_I27', 'Diag_10_J98', 'Diag_9_9971', 'Pro_0331', 'Diag_10_K29', 'Pro_0741', 'Diag_10_L03', 'Diag_10_S27']
len(iii_acols)

342

In [18]:
print(set(iii_acols)-set(dpm_cols))

set()


In [17]:
print(set(dpm_cols)-set(iii_acols))

{'Diag_10_A419', 'Diag_10_R6521', 'Diag_10_Z794', 'Diag_10_D62'}


In [15]:
# 删除 dpm 全部为空的行
Final_df = Final_df[~(Final_df[dpm_cols].isna().all(axis=1))]
Final_df.shape

(67207, 373)

In [19]:
## 填补缺失
Final_df[dpm_cols] = Final_df[dpm_cols].fillna(0)

In [20]:
Final_df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ADMITTIME,INTIME,DISCHTIME,OUTTIME,GENDER,DOB,DOD,...,Med_Potassium Chloride,Med_Prednisone,Med_Propofol,Med_Ranitidine,Med_Simvastatin,Med_Sodium Bicarbonate,Med_Tacrolimus,Med_Thiamine,Med_Vancomycin,Med_Warfarin
0,10000690,25860671,37081114,2150-11-02 18:02:00,2150-11-02 19:37:00,2150-11-12 13:45:00,2150-11-06 17:03:17,F,2064-01-01 00:00:00,2152-01-30 00:00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001217,24597018,37067082,2157-11-18 22:56:00,2157-11-20 19:18:02,2157-11-25 18:00:00,2157-11-21 22:08:00,F,2102-01-01 00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10001725,25563031,31205490,2110-04-11 15:08:00,2110-04-11 15:52:22,2110-04-14 15:00:00,2110-04-12 23:59:56,F,2064-01-01 00:00:00,NaN,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:20:05,2131-01-20 05:15:00,2131-01-20 08:27:30,F,2054-01-01 00:00:00,2131-01-20 00:00:00,...,0.0,40.0,1200.0,300.0,0.0,0.0,0.0,100.0,2750.0,0.0
4,10002013,23581541,39060235,2160-05-18 07:45:00,2160-05-18 10:00:53,2160-05-23 13:30:00,2160-05-19 17:33:33,F,2103-01-01 00:00:00,NaN,...,20.0,0.0,1000.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
Final_df.to_csv('D:/2025UTI/IV/MIMIC4_p_1.csv',index=False)